## **CANOPY-Phenology Model v0.99 in Jupyter Notebook format using Python v3.12**

**The base phenology modeling algorithm was originally developed in 2018-2019 by:**

Matthew Garcia, Ph.D.  
Postdoctoral Research Associate  
Dept. of Forest & Wildlife Ecology  
University of Wisconsin - Madison  
email: matt.e.garcia@gmail.com

The base algorithm was extracted and developed from methods presented in my Ph.D. Dissertation:

> Garcia, M., 2018: *Climatology and Forest Phenology during 1984-2013 around Western Lake Superior, USA*. Ph.D. Dissertation, College of Agriculture and Life Sciences, University of Wisconsin–Madison, 263 pp., ISBN 9780355631883, ProQuest document ID 2013767659.

PI: Prof. Philip A. Townsend (ptownsend@wisc.edu)

---

**CANOPY-Phenology model structural development was continued in 2022-2024 by:**

Matthew Garcia, Ph.D.  
Research Scientist I  
Dept. of Forest & Wildlife Ecology  
University of Wisconsin - Madison  
email: matt.e.garcia@gmail.com

This code is made available under the GNU General Public License (GPL) v3.0. A copy of the GPLv3 is available from the Free Software Foundation at http://www.gnu.org/licenses/gpl.html.

Users are asked to link to this GitHub repository in any derivative and/or published works. Users are expected to reference the attached publication in any derivative and/or published works. As of 31 January 2026, the preferred citation is:

> Garcia, M., and P.A. Townsend, in prep.: "CANOPY: Modeling seasonal and interannual temperate forest phenological variability using Landsat and weather observations." Manuscript in preparation for *MethodsX* with an accompanying paper in *Remote Sensing of Environment*.

---

**CANOPY is a backronym that stands for:**

**C**onditioned  
**A**nalysis of  
**N**ormalized  
**O**bservations in  
**P**henological  
**Y**ears  

It comes from our nickname for the UW–Madison and USDA Forest Service project under which this phase of model development was completed.

CANOPY Project co-PIs during 2022-2026 included:
- Dr. Brian Sturtevant at the USDA Forest Service, Northern Research Station, Rhinelander, WI (brian.r.sturtevant@usda.gov)  
- Dr. Therese Poland at the USDA Forest Service, Northern Research Station, Lansing, MI (therese.poland@usda.gov)  
- Dr. Philip Townsend at the University of Wisconsin – Madison, Madison, WI (ptownsend@wisc.edu)  

**This is the CANOPY-Phenology algorithm. Sometime soon we hope to have a follow-on project to build the CANOPY-Disturbance algorithm...**

---

**This CANOPY-Phenology model is oriented on phenological analysis and prediction on a single-pixel basis.**

This notebook contains the basic functionality of the model:
- fitting a custom-built long-term mean phenological curve against a single Landsat vegetation index (VI)
- extraction of mean phenological curve indicators and metrics
- PLS regression on climatological anomalies against the Landsat VI
- PLS prediction of the Landsat VI in analyzed and forecast periods on an image basis
- outlier detection using a forecast-interval approach
- iterated outlier removal and re-modeling until an optimum model is obtained

A separate notebook now handles the post-modeling procedure for daily VI prediction:
- PLS prediction on a *daily* (instead of image) basis in user-specified analysis and forecast periods

This notebook version is heavily modified from my original dissertation algorithm. Model version 0.99 already exceeds my dissertation model with the following features:
- outlier detection (and PLS re-modeling) on the selected Landsat VI, rather than the KTTC components
- automatic selection of PLS model components using minimum AICc (to prevent model overfitting)
- calculation of variable importance in projection (VIP) in the PLS regression procedure
- PLS variable reduction using the VIP metric (instead of a computationally intensive progressive elimination procedure)
- image-based Landsat VI forecasting and evaluation metrics for a forecast (or other non-analysis) period
- outlier detection using the predicted VI in the analysis period via prediction interval
- iterated outlier removal and VI re-modeling until the optimum model (at minimum AICc) is obtained
- daily Landsat VI prediction in the growing season on an annual basis for a user-specified year(s) in the analysis and/or forecast period
- fitting the custom-built phenological curve against the predicted VI time series on an annual basis for the extraction of individual-year seasonal indicators and metrics

This notebook is intended for testing and experiments on a single-pixel basis and is released with sample datasets as an open-source digital supplement to the above publication(s).

Eventually, I intend to return some original (dissertation) functionality:
- PLS regression and prediction on multiple Landsat VIs (other than KTTC components)
- PLS regression and prediction on KTTC components (requires a different type of fitted phenological curve)
- using climatological frost dates (mean and stdev) to specify VI curve fit SOS/EOS bounds

Possible additional near-future version functionality may also include:
- PLS regression and prediction on Landsat surface reflectance bands, instead of calculated VIs

**This is a model intended for application to a *single Landsat pixel* using available observations.**

**This procedure applies *no* spatial analysis at *any* scale to the model results.**


---

### **Model Setup**

**Notebook Cell 1**: load python libraries and set some notice and display parameters

In [ ]:
# notebook uses python standard libraries and common additional libraries
#

#
# standard libraries, numpy, pandas, matplotlib, seaborn, scipy
import os
import sys
import platform
import csv
from copy import deepcopy
import ast
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.stats import linregress, kstest
from scipy.optimize import curve_fit

#
# PLS regression and modeling via scikit-learn library
import sklearn
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import train_test_split

#
# suppress warnings about fragmented pandas DataFrames
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

#
# suppress warnings about all the open figures
plt.rcParams.update({'figure.max_open_warning': 0})

#
# magic function to view graphs in notebook, instead of pop-up windows
%matplotlib inline

**Notebook Cell 2**: check current versions of python libraries

In [ ]:
print('Python package versions')
print(f'    python v. {platform.python_version()}')
print(f'    numpy v. {np.__version__}')
print(f'    pandas v. {pd.__version__}')
print(f'    matplotlib v. {mpl.__version__}')
print(f'    seaborn v. {sns.__version__}')
print(f'    scipy v. {scipy.__version__}')
print(f'    sklearn v. {sklearn.__version__}')

---

**Notebook Cell 3**: establish the WxCD and VI variables available (based on established pre-processing methods, which can change)

In [ ]:
# contents of phenology_model_vars.py
#
# NOTE as of v0.99 only the variable "wxcd_std_anom_vars" gets used outside of this cell
#


periods_days = [1, 3, 5, 10, 15, 30, 45, 60, 90, 120, 180, 270, 365]
wxcd_base_vars = ['prcp', 'tavg', 'tmax', 'tmin', 'trange']
wxcd_xdd_vars = ['tavg_cd', 'tavg_cdd', 'tavg_fd', 'tmin_fd',
                 'tavg_gdd_base0', 'tavg_gdd_base5', 'tavg_gdd_base10']
#
Daymet_vars = []
wxcd_std_anom_vars = []
for v in wxcd_base_vars:
    for p in periods_days:
        if v == 'prcp':
            Daymet_vars.append(f'{v}_{p}d_sum')
        else:
            Daymet_vars.append(f'{v}_{p}d_avg')
            Daymet_vars.append(f'{v}_{p}d_std')
        Daymet_vars.append(f'{v}_{p}d_clim_avg')
        Daymet_vars.append(f'{v}_{p}d_clim_std')
        Daymet_vars.append(f'{v}_{p}d_std_anom')
        wxcd_std_anom_vars.append(f'{v}_{p}d_std_anom')
#
for v in wxcd_xdd_vars:
    Daymet_vars.append(v)
    Daymet_vars.append(f'{v}_clim_avg')
    Daymet_vars.append(f'{v}_clim_std')
    Daymet_vars.append(f'{v}_std_anom')
    wxcd_std_anom_vars.append(f'{v}_std_anom')
#
Landsat_vars = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2',
                'SR', 'MSI', 'NDVI', 'EVI', 'SAVI', 'NDMI', 'NBR',
                'BGT', 'GRN', 'WET']
kttc_vars = ['BGT', 'GRN', 'WET']


#
# end contents of phenology_model_vars.py

---

### **Model Input**

**Notebook Cell 4**: location ID, where to find its input data, and which VI to use

In [ ]:
# required input to model_vi_phenology.py
#
# FUTURE WORK: convert to argparse usage for ALL metavals
#


point_id = '2023-120_CC'
input_path = f'{point_id}_input'
vi_name = 'EVI'


#
# end required input to model_vi_phenology.py

**Notebook Cell 5**: configurable values used throughout the model procedure

In [ ]:
# part of phenology_model_input.py
#
# metadata setup
#
# NOTE as of v0.99 there are switches for exploratory analyses and graph output (plots)
#
# FUTURE WORK: convert to argparse usage for ALL metavals
#


metavals = {'input_path': input_path,
            'output_path': f'{point_id}_{vi_name.upper()}_analyses',
            'point_id': point_id,
            'daymet_input_fname': f'{point_id}_Daymet_data.csv',
            'landsat_input_fname': f'{point_id}_Landsat_SR_VI.csv',
            'n_observations': 0,
            'x_basis': 'DOY',
            'vi_name': vi_name.upper(),
            'kttc_curve_type': 1,  # placeholder for return of KTTC processing
            'kttc_curve_str': 'SIN',  # placeholder for return of KTTC processing
            'vi_curve_type': 4,
            'vi_curve_str': 'ABC',
            'doy_min': 1,  # DOY
            'doy_max': 365,  # DOY
            'growing_season_start': 80,  # DOY of spring equinox
            'growing_season_end': 355,  # DOY of winter solstice
            'data_begin_year': 1984,
            'data_end_year': 2023,
            'analysis_begin_year': 1984,
            'analysis_end_year': 2023,
            'forecast_begin_year': 0,
            'forecast_end_year': 0,
            'nc_max': 20,
            'test_split': 0.2,
            'n_iterations': 500,
            'vip_threshold': 1.0,
            'remove_outliers': True,
            'outlier_confidence': 0.95,
            'outlier_round': 0,
            'max_outlier_rounds': 3,
            'n_outliers': 0,
            'graph_output': True,
            'exploratory': True}


#
# end part of phenology_model_input.py

---

**Notebook Cell 6**: read the input dataset(s) for the location to be modeled, merge the VI and WxCD data, and do some initial clean-up and trimming

In [ ]:
# part of model_vi_phenology.py
#
# primary dataframe setup and initial reporting
#


print(f'Analyzing Landsat and WxCD data at {point_id}')
#
# if output path doesn't exist, make it
output_path = metavals['output_path']
if not os.path.exists(output_path):
    print(f'- making output path {output_path}')
    os.makedirs(output_path)
#
# read Landsat input data
input_path = metavals['input_path']
landsat_input_fname = metavals['landsat_input_fname']
landsat_input_fpath = f'{input_path}/{landsat_input_fname}'
print(f'- reading Landsat input file {landsat_input_fname}')
vi_input_df = pd.read_csv(landsat_input_fpath, index_col=None)
#
# read Daymet input data
daymet_input_fname = metavals['daymet_input_fname']
daymet_input_fpath = f'{input_path}/{daymet_input_fname}'
print(f'- reading Daymet input file {daymet_input_fname}')
wxcd_input_df = pd.read_csv(daymet_input_fpath, index_col=None)
print()
#
# merge input DataFrames
vi_input_df.drop(columns=['QA_condition'], inplace=True)  # all input dates are 'clear'
year = vi_input_df['Year']
month = vi_input_df['Month']
day = vi_input_df['Day']
vi_input_df['Date'] = [f'{yyyy}{mm:02d}{dd:02d}' for yyyy, mm, dd in zip(year, month, day)]
doy = vi_input_df['DOY']
vi_input_df['DYear'] = [float(yyyy + (d/366)) for yyyy, d in zip(year, doy)]
wxcd_input_df.drop(columns=['point_id', 'Year', 'DOY'], inplace=True)
wxcd_input_df.fillna(0, inplace=True)
wxcd_vi_input_df = pd.merge(vi_input_df, wxcd_input_df, on='YYYY_DOY')
n_input_data_rows = len(wxcd_vi_input_df)
data_begin_year = metavals['data_begin_year']
data_end_year = metavals['data_end_year']
print(f'Data for {point_id} span {data_begin_year}-{data_end_year} with {n_input_data_rows} rows')
#
# clean DataFrame of rows with missing data
wxcd_vi_all_df = wxcd_vi_input_df.dropna()
n_data_rows = len(wxcd_vi_all_df)
if n_data_rows < n_input_data_rows:
    print(f'- removing missing data leaves {n_data_rows} rows')
#
# trim DataFrame to analysis period (if different from data coverage period)
if (metavals['data_begin_year'] == metavals['analysis_begin_year']) and \
        (metavals['data_end_year'] == metavals['analysis_end_year']):
    print(f'- using all available years for analysis ({n_data_rows} rows)')
    wxcd_vi_model_df = deepcopy(wxcd_vi_all_df)
else:
    wxcd_vi_model_df = wxcd_vi_all_df[wxcd_vi_all_df['Year'] >= metavals['analysis_begin_year']]
    wxcd_vi_model_df = wxcd_vi_model_df[wxcd_vi_model_df['Year'] <= metavals['analysis_end_year']]
    print(f'- trimming data to specified analysis years leaves {len(wxcd_vi_model_df)} rows')
#
# trim DataFrame to growing season
wxcd_vi_df = wxcd_vi_model_df[wxcd_vi_model_df['DOY'] >= metavals['growing_season_start']]
wxcd_vi_df = wxcd_vi_df[wxcd_vi_df['DOY'] <= metavals['growing_season_end']]
print(f'- trimming to specified growing season DOY range leaves {len(wxcd_vi_df)} rows')
#
# get n_observations
metavals['n_observations'] = len(wxcd_vi_df)
#
# save input DataFrame
outfname = f'{point_id}_input_WxCD_VI_cleaned_trimmed.csv'
outfpath = f'{output_path}/{outfname}'
wxcd_vi_df.to_csv(outfpath, index=False)
print(f'- saved {outfname}')
print()


#
# end part of model_vi_phenology.py

---

### **Standard Seasonal Phenology Model and Metrics**

<p>The phenological curve fitted to the user's selected Landsat VI is described by a 7-parameter piecewise-continuous function using two halves of a cosine curve (for the green-up and senescence portions) and a linear function with green-down effect (for the mature period). There is some additional math to help smooth the curve at the junctions. The ordinary least-squares (OLS) fit is based on the entire drawn curve, not on individual pieces of the curve. The resulting curve was illustrated in figure 4.2 (p. 58) of my dissertation, based on fitting an actual single-pixel Landsat time series:</p>

<img src="./phenology_model_curve_indicators_schematic.png" alt="Phenological curve indicators" align="center" width="600"/>

<p>All of the indicators and metrics for the fitted curve are calculated and reported below. This curve is approximately valid for almost all forest types, even evergreens, and the formulation allows a mature period that is flat or even reverse-slope (maximum VI just before senescence, instead of just after green-up) if that is found to fit better. I am working to re-implement two constraints on the curve, specifically on the earliest SOS and latest EOS, using climatological spring and autumn frost dates, respectively. That requires further analysis than I have completed thus far in my climatological reprocessing work. For now, the open values for SOSmin and EOSmax provided below seem to work well.</p>

<p>NOTE that this curve is *not* appropriate for some VIs, such as the Tasseled Cap Greenness (GRN) and Wetness (WET). I plan to restore a fitted sine curve function that I've used before for those as part of this process, and the user will choose which curve type gets fitted to their desired VI.</p>

**Notebook Cell 7**: calculate numerous error metrics for a fitted phenological curve

In [ ]:
# contents of phenology_model_error_metrics.py
#


def calc_fit_error_metrics(n_obs, n_params, values, residuals):
    bias = np.mean(residuals)
    mae = np.mean(np.abs(residuals))
    rmse = np.sqrt(np.mean(residuals**2))
    sse = np.sum(residuals**2)
    values_mean = np.mean(values)
    sst = np.sum((values - values_mean)**2)
    rsq = 1 - (sse / sst)
    #
    residuals_mean = np.mean(residuals)
    residuals_var = np.var(residuals)
    ssd = np.sum((residuals - residuals_mean)**2)
    #
    # formula for log-L from Wikipedia:
    #     https://en.wikipedia.org/wiki/Maximum_likelihood_estimation
    # assuming residuals have a normal distribution
    #
    log_L = -1 * (n_obs / 2) * np.log(2 * np.pi * residuals_var) - (ssd / (2 * residuals_var))
    aic = 2.0 * (n_params - log_L)
    if (n_obs - n_params) > 2:
        aicc = aic + (2 * (n_params + 1) * (n_params + 2) / (n_obs - n_params - 2))
    else:
        aicc = aic
    bic = (n_params * np.log(n_obs)) - (2.0 * log_L)
    #
    fit_error_metrics = [bias, mae, rmse, sse, rsq, aic, aicc, bic]
    #
    return fit_error_metrics


#
# end contents of phenology_model_error_metrics.py

**Notebook Cell 8**: definition and implementation of the ABC curve fit procedure

Calls function in **Notebook Cell 7**

In [ ]:
# contents of phenology_model_fit_abc_curve.py
#


#
# parameters of the ABC curve
abc_curve_fit_vars = ['Ymin', 'Ymin_err',
                      'SOS', 'SOS_err',
                      'Ymax_a', 'Ymax_a_err',
                      'Ymax_a_doy', 'Ymax_a_doy_err',
                      'Ymax_b', 'Ymax_b_err',
                      'Ymax_b_doy', 'Ymax_b_doy_err',
                      'EOS', 'EOS_err',
                      'Bias', 'MAE', 'RMSE', 'SSE', 'Rsq',
                      'AIC', 'AICc', 'BIC']


#
# mathematical definition of the ABC curve
def abc_fn(X_sample, Ymin, SOS, Ymax_a, Ymax_a_X, Ymax_b, Ymax_b_X, EOS):
    """
    builds a piecewise continuous asymmetric broken cosine phenological curve
    1. fit a cosine curve to the data (x = 0 at mid-summer)
    2. break it at the peak and separate the halves
    3. allow the halves to have different parameters (period & amplitude)
    """
    Xmin = int(X_sample[-2])
    Xmax = int(X_sample[-1])
    X_sample = np.array(X_sample[:-2]).astype(int)
    X_offset = 10
    X = np.arange(Xmin, Xmax + 1)
    Y = np.zeros(np.shape(X))
    #
    # in this loop, draw most of the phenological curve
    # - Y = Ymin for pre- and post-season periods
    # - Y = cosine curves for green-up and senescence periods
    # - leave mature period for quasi-adaptive method below
    #
    for yt, xt in enumerate(X):
        if xt < SOS:                                                          # pre-season phenophase
            Y[yt] = Ymin
        elif (xt >= SOS) and (xt < (Ymax_a_X + X_offset)):                    # green-up phenophase
            amp = (Ymax_a - Ymin) / 2.0
            xt_rad = ((xt - Ymax_a_X) / float(Ymax_a_X - SOS)) * np.pi
            Y[yt] = Ymin + amp * (1 + np.cos(xt_rad))
        elif (xt >= (Ymax_a_X + X_offset)) and (xt < (Ymax_b_X - X_offset)):  # mature phenophase
            pass  # see below
        elif (xt >= (Ymax_b_X - X_offset)) and (xt < EOS):                    # senescence phenophase
            amp = (Ymax_b - Ymin) / 2.0
            xt_rad = ((xt - Ymax_b_X) / float(EOS - Ymax_b_X)) * np.pi
            Y[yt] = Ymin + amp * (1 + np.cos(xt_rad))
        elif xt >= EOS:                                                       # post-season phenophase
            Y[yt] = Ymin
    #
    # quasi-adaptive method for smoother curves around the mature transitions
    # - matches mature phenophase slope with the green-up and senescence limbs
    #
    slope = (Ymax_b - Ymax_a) / float(Ymax_b_X - Ymax_a_X)
    if slope > 0:
        for xt_a in range(int(round(Ymax_a_X) - X_offset),
                          int(round(Ymax_a_X) - 1)):
            slope_a = Y[xt_a + 1] - Y[xt_a]
            if slope_a <= slope:
                break
        for xt_b in range(int(round(Ymax_b_X) - X_offset + 1),
                          int(round(Ymax_b_X))):
            slope_b = Y[xt_b] - Y[xt_b - 1]
            if slope_b <= slope:
                break
        slope = (Y[xt_b] - Y[xt_a]) / float(xt_b - xt_a)
        for yt in range(xt_a, xt_b):
            Y[yt] = Y[xt_a] + slope * (yt - xt_a)
    elif slope < 0:
        for xt_a in range(int(round(Ymax_a_X)),
                          int(round(Ymax_a_X) + X_offset - 1)):
            slope_a = Y[xt_a + 1] - Y[xt_a]
            if slope_a <= slope:
                break
        for xt_b in range(int(round(Ymax_b_X)),
                          int(round(Ymax_b_X) + X_offset - 1)):
            slope_b = Y[xt_b + 1] - Y[xt_b]
            if slope_b <= slope:
                break
        slope = (Y[xt_b] - Y[xt_a]) / float(xt_b - xt_a)
        for yt in range(xt_a, xt_b):
            Y[yt] = Y[xt_a] + slope * (yt - xt_a)
    else:
        for yt in range(int(round(Ymax_a_X)), int(round(Ymax_b_X))):
            Y[yt] = Ymax_a
    #
    # return curve values at only the sample dates
    Y_fit = np.zeros(len(X_sample))
    for d, doy in enumerate(X_sample):
        Y_fit[d] = Y[doy - 1]
    #
    return Y_fit


#
# construct an ABC curve from known parameter values
def get_abc_curve(x_range, params):
    fit_y = abc_fn(x_range, params[0], params[2], params[4],
                   params[6], params[8], params[10], params[12])
    #
    return fit_y


#
# fit an ABC curve to the given data and report parameter values and error metrics
def optimize_fit_abc(X, Y, X_min, X_max):
    #
    # initial parameter values
    p_init = [0.0, 90, 1.0, 120, 1.0, 240, 270]
    #
    # parameter bounds for fit process
    lower_bounds = [-1.0, 1, -1.0, 1, -1.0, 228, 228]
    upper_bounds = [1.0, 227, 1.0, 227, 1.0, 365, 365]
    #
    # sneak the overall bounds of X into the curve_fit argument structure
    X_ext = np.append(X, [X_min, X_max])
    #
    # bounded ordinary least-squares (OLS) curve fit process via scipy.optimize.curve_fit
    p_opt, p_cov = curve_fit(abc_fn, X_ext, Y, p0=p_init,
                             bounds=(lower_bounds, upper_bounds),
                             method='trf', max_nfev=1E6)
    #
    # fitted curve error metrics
    p_err = np.sqrt(np.diag(p_cov))
    pY = abc_fn(X_ext, p_opt[0], p_opt[1], p_opt[2],
                p_opt[3], p_opt[4], p_opt[5], p_opt[6])
    Y_res = pY - Y
    #
    fit_error_metrics = calc_fit_error_metrics(len(X), len(p_opt), Y, Y_res)
    #
    return p_opt, p_err, fit_error_metrics


#
# entry to the curve fit procedure
def pheno_fit_abc(xvals, yvals, xmin=1, xmax=365):
    fit, fit_err, fit_error_metrics = optimize_fit_abc(xvals, yvals, xmin, xmax)
    pheno_params = [fit[0], fit_err[0], fit[1], fit_err[1],
                    fit[2], fit_err[2], fit[3], fit_err[3],
                    fit[4], fit_err[4], fit[5], fit_err[5],
                    fit[6], fit_err[6]]
    pheno_params += fit_error_metrics
    x_range = list(np.arange(xmin, xmax + 1))
    x_range_ext = x_range + [xmin, xmax]
    fit_all_y = abc_fn(x_range_ext, fit[0], fit[1], fit[2],
                       fit[3], fit[4], fit[5], fit[6])
    fit_sample_y = np.array([fit_all_y[int(val) - 1] for val in xvals])
    pheno_anom = yvals - fit_sample_y
    #
    return pheno_params, pheno_anom


#
# end contents of phenology_model_fit_abc_curve.py

---

**Notebook Cell 9**: plot VI values by decimal year

In [ ]:
# not yet part of phenology_model_plots.py
#


def plot_vi_by_dyear(vi_name, vi_dyears, vi_values, begin_year, end_year,
                     outlier_round, output_path, fname_tuple):
    #
    fig = plt.figure(figsize=(12,4))
    #
    # plot VI values by DYear
    ax = plt.subplot(1, 1, 1)
    plt.scatter(vi_dyears, vi_values, marker='x', c='k', s=30)
    plt.xlim([begin_year, end_year+1])
    years = np.arange(begin_year, end_year+2)
    even_years = [year if year % 2 == 0 else '' for year in years]
    plt.xticks(years, fontsize=10)
    ax.set_xticklabels(even_years, fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('Year', fontsize=11)
    plt.ylabel(vi_name, fontsize=11)
    title = f'{begin_year}-{end_year} {vi_name} observations '
    title += f'(round {outlier_round}, n = {len(vi_values)})'
    plt.title(title, fontsize=12)
    #
    plt.tight_layout()
    plotfname = '%s_%s_%s_%s_round_%d_obs_by_dyear.png' % fname_tuple
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    #
    return


**Notebook Cell 10**: plot VI values by observation day-of-year

In [ ]:
# derived from part of phenology_model_plots.py
#


def plot_vi_by_doy(vi_name, vi_doys, vi_values, doys_range,
                   season_start, season_end, begin_year, end_year,
                   outlier_round, output_path, fname_tuple, vi_outliers_df):
    #
    fig = plt.figure(figsize=(6, 4))
    #
    # plot VI values by DOY
    ax = plt.subplot(1, 1, 1)
    plt.scatter(vi_doys, vi_values, marker='x', c='k', s=30)
    if len(vi_outliers_df):
        outliers_doys = np.array(vi_outliers_df['DOY'])
        outliers_values = np.array(vi_outliers_df[vi_name])
        plt.scatter(outliers_doys, outliers_values, marker='o', c='r',
                    edgecolor='k', s=30)
    plt.xlim([season_start, season_end])
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('DOY', fontsize=11)
    plt.ylabel(vi_name, fontsize=11)
    title = f'{begin_year}-{end_year} {vi_name} observations '
    title += f'(round {outlier_round}, n = {len(vi_values)})'
    plt.title(title, fontsize=12)
    #
    plt.tight_layout()
    plotfname = '%s_%s_%s_%s_round_%d_obs_by_doy.png' % fname_tuple
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    #
    return


**Notebook Cell 11**: plot fitted phenological curve along with VI values by day-of-year, with outliers (if any) marked

In [ ]:
# derived from part of phenology_model_plots.py
#


def plot_vi_curve_fit(vi_name, vi_doys, vi_values, full_year_mean_vi, 
                      rsq, doys_range, season_start, season_end,
                      begin_year, end_year, var_fit_params, outlier_round,
                      output_path, fname_tuple, vi_outliers_df):
    #
    fig = plt.figure(figsize=(6, 4))
    #
    # plot VI values by DOY
    ax = plt.subplot(1, 1, 1)
    plt.scatter(vi_doys, vi_values, marker='x', c='k', s=30)
    if len(vi_outliers_df):
        outliers_doys = np.array(vi_outliers_df['DOY'])
        outliers_values = np.array(vi_outliers_df[vi_name])
        plt.scatter(outliers_doys, outliers_values, marker='o', c='r',
                    edgecolor='k', s=30)
    plt.plot(doys_range, full_year_mean_vi, 'b-', linewidth=2)
    control_doys = [var_fit_params[2], var_fit_params[6],
                    var_fit_params[10], var_fit_params[12]]
    control_vi_values = [full_year_mean_vi[int(round(var_fit_params[2])) - 1],
                         var_fit_params[4], var_fit_params[8], 
                         full_year_mean_vi[int(round(var_fit_params[12])) - 1]]
    plt.scatter(control_doys, control_vi_values, marker='o', c='g', edgecolor='g', s=50)
    plt.xlim([season_start, season_end])
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('DOY', fontsize=11)
    plt.ylabel(vi_name, fontsize=11)
    title = f'{begin_year}-{end_year} {vi_name} mean phenocurve '
    title += f'(round {outlier_round}, Rsq = {rsq:.3f})'
    plt.title(title, fontsize=12)
    #
    plt.tight_layout()
    plotfname = '%s_%s_%s_%s_round_%d_obs+phenocurve.png' % fname_tuple
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    #
    return


**Notebook Cell 12**: subtract fitted phenological curve from VI values by day-of-year and plot residuals, with outliers (if any) marked

In [ ]:
# derived from part of phenology_model_plots.py
#


def plot_vi_residuals(vi_name, vi_doys, vi_fit_residuals, full_year_mean_vi, 
                      doys_range, season_start, season_end,
                      begin_year, end_year, outlier_round,
                      output_path, fname_tuple, vi_outliers_df):
    #
    fig = plt.figure(figsize=(6, 4))
    #
    # plot VI values by DOY
    ax = plt.subplot(1, 1, 1)
    plt.scatter(vi_doys, vi_fit_residuals, marker='x', c='k', s=30)
    if len(vi_outliers_df):
        outliers_doys = np.array(vi_outliers_df['DOY'])
        outliers_values = np.array(vi_outliers_df[vi_name])
        curve_values = np.array([full_year_mean_vi[int(doy) - 1]
                                 for doy in outliers_doys])
        outliers_residuals = outliers_values - curve_values
        plt.scatter(outliers_doys, outliers_residuals, marker='o',
                    c='r', edgecolor='k', s=30)
    plt.plot(doys_range, np.zeros_like(full_year_mean_vi), 'b-', linewidth=2)
    plt.xlim([season_start, season_end])
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('DOY', fontsize=11)
    plt.ylabel(f'{vi_name} residual', fontsize=11)
    title = f'{begin_year}-{end_year} {vi_name} residuals (round {outlier_round})'
    plt.title(title, fontsize=12)
    #
    plt.tight_layout()
    plotfname = '%s_%s_%s_%s_round_%d_residuals.png' % fname_tuple
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    #
    return


**Notebook Cell 13**: plot a histogram of the VI residuals from **Notebook Cell 12**

In [ ]:
# derived from part of phenology_model_plots.py
#


def plot_vi_residuals_histogram(vi_name, vi_fit_residuals, begin_year, end_year,
                                ks_value, ks_pvalue, outlier_round, output_path,
                                fname_tuple, vi_outliers_df):
    #
    fig = plt.figure(figsize=(6, 4))
    #
    # plot VI values by DOY
    ax = plt.subplot(1, 1, 1)
    nbins = int(round((np.max(vi_fit_residuals) - np.min(vi_fit_residuals)) / 0.01))
    plt.hist(vi_fit_residuals, bins=nbins)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel(f'{vi_name} residuals', fontsize=11)
    plt.ylabel('count', fontsize=11)
    title = f'{begin_year}-{end_year} {vi_name} residuals '
    title += f'(round {outlier_round}, K-S test = {ks_value:.3f}, p = {ks_pvalue:.3f})'
    plt.title(title, fontsize=12)
    #
    plt.tight_layout()
    plotfname = '%s_%s_%s_%s_round_%d_residuals_histogram.png' % fname_tuple
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    #
    return


---

### **Model Procedure for Fitting the Mean Phenological Curve**

**Notebook Cell 14**: Fit the long-term mean phenological curve to the selected VI observations and plot/save the results.

Calls functions in **Notebook Cells 8, 9, 10, 11, and 12**

In [ ]:
# contents of phenology_model_vi.py
#


def model_vi_pheno_curve(metavals, vi_retained_df, vi_outliers_df=pd.DataFrame()):
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    x_basis = metavals['x_basis']
    curve_str = metavals['vi_curve_str']
    curve_fit_vars = abc_curve_fit_vars
    begin_year = metavals['analysis_begin_year']
    end_year = metavals['analysis_end_year']
    outlier_round = metavals['outlier_round']
    remove_outliers = metavals['remove_outliers']
    n_outliers = metavals['n_outliers']
    doy_min = metavals['doy_min']
    doy_max = metavals['doy_max']
    growing_season_start = metavals['growing_season_start']
    growing_season_end = metavals['growing_season_end']
    graph_output = metavals['graph_output']
    fname_tuple = (point_id, vi_name, curve_str, x_basis, outlier_round)
    #
    #
    print(f'Round {outlier_round}: {n_outliers} outliers identified')
    print(f'Step 0: fit {curve_str} phenology curve to {vi_name}')
    print()
    #
    # initialize table for VI curve fit parameters
    pheno_fit_params_df = pd.DataFrame({'parameter': curve_fit_vars})
    #
    # initialize table for fitted curve VI anomalies
    # note that only the "retained" data are used, with no outliers present
    pheno_fit_residuals_df = pd.DataFrame({'Date': np.array(vi_retained_df['Date']),
                                           'DYear': np.array(vi_retained_df['DYear']),
                                           'DOY': np.array(vi_retained_df['DOY'])})
    #
    # get DYear, DOY, and VI values
    vi_dyears = np.array(vi_retained_df['DYear'])
    vi_doys = np.array(vi_retained_df[x_basis]).astype(int)
    vi_values = np.array(vi_retained_df[vi_name])
    #
    # plot VI observations by DYear
    if graph_output:
        plot_vi_by_dyear(vi_name, vi_dyears, vi_values, begin_year, end_year,
                         outlier_round, output_path, fname_tuple)
    print()
    #
    # fit VI mean phenology curve
    print(f'Fitting {vi_name} by {x_basis}')
    vi_fit_params, vi_fit_residuals = pheno_fit_abc(vi_doys, vi_values, doy_min, doy_max)
    vi_doys_ext = np.append(vi_doys, [doy_min, doy_max])
    vi_fitted = get_abc_curve(vi_doys_ext, vi_fit_params)
    pheno_fit_params_df[f'{vi_name}_by_{x_basis}'] = vi_fit_params
    pheno_fit_residuals_df[vi_name] = vi_values
    pheno_fit_residuals_df[f'{vi_name}_fitted'] = vi_fitted
    pheno_fit_residuals_df[f'{vi_name}_residual'] = vi_fit_residuals
    rsq = vi_fit_params[curve_fit_vars.index('Rsq')]
    print(f'- Rsq = {rsq:.3f}')
    print()
    #
    # get full-year daily VI values from fit parameters
    doys_range = np.arange(doy_min, doy_max + 1)
    doys_range_ext = np.append(doys_range, [doy_min, doy_max])
    full_year_mean_vi = get_abc_curve(doys_range_ext, vi_fit_params)
    #
    # plot VI obervations by DOY with marked outliers
    if graph_output:
        plot_vi_by_doy(vi_name, vi_doys, vi_values, doys_range,
                       growing_season_start, growing_season_end, begin_year, end_year,
                       outlier_round, output_path, fname_tuple, vi_outliers_df)
    #
    # plot VI obervations by DOY with marked outliers and fitted curve, and VI residuals
    if graph_output:
        plot_vi_curve_fit(vi_name, vi_doys, vi_values, full_year_mean_vi, 
                          rsq, doys_range, growing_season_start, growing_season_end,
                          begin_year, end_year, vi_fit_params, outlier_round,
                          output_path, fname_tuple, vi_outliers_df)
    #
    # plot VI residuals by DOY with marked outliers
    if graph_output:
        plot_vi_residuals(vi_name, vi_doys, vi_fit_residuals, full_year_mean_vi, 
                          doys_range, growing_season_start, growing_season_end,
                          begin_year, end_year, outlier_round,
                          output_path, fname_tuple, vi_outliers_df)
    #
    # Check VI residuals for normal distribution using scipy.stats.kstest
    ks_statistic, ks_pvalue = kstest(vi_fit_residuals, 'norm')
    print('Kolmogorov-Smirnov test for normality of residuals')
    print(f'- statistic = {ks_statistic:.3f}')
    print(f'- p = {ks_pvalue:.3f}')
    print()
    #
    # plot VI residuals histogram
    if graph_output:
        plot_vi_residuals_histogram(vi_name, vi_fit_residuals, begin_year, end_year,
                                    ks_statistic, ks_pvalue, outlier_round, output_path,
                                    fname_tuple, vi_outliers_df)
    print()
    #
    # save VI curve fit parameters to CSV
    outfname = '%s_retained_%s_%s_%s_round_%d_fit_params.csv' % fname_tuple
    outfpath = f'{output_path}/{outfname}'
    pheno_fit_params_df.to_csv(outfpath, index=False)
    print(f'saved {outfname}')
    #
    # save retained VI data with curve-based anomalies to CSV
    outfname = '%s_retained_%s_%s_%s_round_%d_fit_residuals.csv' % fname_tuple
    outfpath = f'{output_path}/{outfname}'
    pheno_fit_residuals_df.to_csv(outfpath, index=False)
    print(f'saved {outfname}')
    #
    # save full-year mean VI curve to NPY
    outfname = '%s_retained_%s_%s_%s_round_%d_full_year.npy' % fname_tuple
    outfpath = f'{output_path}/{outfname}'
    np.save(outfpath, full_year_mean_vi)
    print(f'saved {outfname}')
    print()
    #
    return pheno_fit_params_df, pheno_fit_residuals_df, full_year_mean_vi


#
# end contents of phenology_model_vi.py

**Notebook Cell 15**: invoke function to fit the long-term mean phenological curve

Calls function in **Notebook Cell 14**

In [ ]:
# part of model_vi_phenology.py
#


pheno_fit_params_df, pheno_fit_residuals_df, full_year_mean_vi = \
    model_vi_pheno_curve(metavals, wxcd_vi_df)
pheno_fit_residuals_orig_df = deepcopy(pheno_fit_residuals_df)


#
# end part of model_vi_phenology.py

**Notebook Cell 16**: statistics to support phenological metric calculations

In [ ]:
# part of phenology_model_stats.py
#


def additive_error(var1_err, var2_err):
    err = np.sqrt(var1_err**2 + var2_err**2)
    #
    return err


def multiplicative_error(var0, var1, var1_err, var2, var2_err):
    err = np.sqrt(var0**2 * ((var1_err / var1)**2 + (var2_err / var2)**2))
    #
    return err


def regress(x, y):
    # uses scipy.stats.linregress
    slope, intercept, corr, sig, stderr = linregress(x, y)
    #
    return [slope, intercept, corr, sig, stderr]


#
# end part of phenology_model_stats.py

**Notebook Cell 17**: phenological metric calculations

Calls functions in **Notebook Cell 16**

In [ ]:
# part of phenology_model_vi_metrics.py
#


def calc_fitted_pheno_curve_metrics(vi_name, curve_params):
    """Calculate indicators and metrics for fitted phenological curve."""
    #
    labels = []
    values = []
    errors = []
    #
    # clearly identify fitted curve indicators
    labels.append(f'min {vi_name}')
    VI_min = curve_params[0]
    values.append(VI_min)
    VI_min_err = curve_params[1]
    errors.append(VI_min_err)
    #
    labels.append(f'{vi_name} SOS [DOY]')
    SOS = curve_params[2]
    values.append(SOS)
    SOS_err = curve_params[3]
    errors.append(SOS_err)
    #
    labels.append(f'Spring max {vi_name}')
    SVI_max = curve_params[4]
    values.append(SVI_max)
    SVI_max_err = curve_params[5]
    errors.append(SVI_max_err)
    #
    labels.append(f'{vi_name} SOM [DOY]')
    SOM = curve_params[6]
    values.append(SOM)
    SOM_err = curve_params[7]
    errors.append(SOM_err)
    #
    labels.append(f'Autumn max {vi_name}')
    AVI_max = curve_params[8]
    values.append(AVI_max)
    AVI_max_err = curve_params[9]
    errors.append(AVI_max_err)
    #
    labels.append(f'{vi_name} EOM [DOY]')
    EOM = curve_params[10]
    values.append(EOM)
    EOM_err = curve_params[11]
    errors.append(EOM_err)
    #
    labels.append(f'{vi_name} EOA [DOY]')
    EOA = curve_params[12]
    values.append(EOA)
    EOA_err = curve_params[13]
    errors.append(EOA_err)
    #
    # calculate fitted-curve-based metrics
    labels.append(f'Spring {vi_name} range')
    SVI_range = SVI_max - VI_min
    values.append(SVI_range)
    SVI_range_err = additive_error(VI_min_err, SVI_max_err)
    errors.append(SVI_range_err)
    #
    labels.append(f'Spring {vi_name} inflection')
    VI_SI = (VI_min + SVI_max) / 2.0
    values.append(VI_SI)
    VI_SI_err = SVI_range_err
    errors.append(VI_SI_err)
    #
    labels.append(f'Spring {vi_name} inflection [DOY]')
    SI = (SOS + SOM) / 2.0
    values.append(SI)
    SI_err = additive_error(SOS_err, SOM_err)
    errors.append(SI_err)
    #
    labels.append(f'Spring {vi_name} duration [d]')
    DOS = SOM - SOS
    values.append(DOS)
    DOS_err = SI_err
    errors.append(DOS_err)
    #
    labels.append(f'Spring {vi_name} slope [1/d]')
    Slope_of_Spring = SVI_range / DOS
    values.append(Slope_of_Spring)
    Slope_of_Spring_err = multiplicative_error(Slope_of_Spring, SVI_range, SVI_range_err,
                                               DOS, DOS_err)
    errors.append(Slope_of_Spring_err)
    #
    labels.append(f'Maturity {vi_name} duration [d]')
    DOM = EOM - SOM
    values.append(DOM)
    DOM_err = additive_error(SOM_err, EOM_err)
    errors.append(DOM_err)
    #
    labels.append(f'Maturity {vi_name} slope [1/d]')
    Green_down = AVI_max - SVI_max
    Green_down_err = additive_error(SVI_max_err, AVI_max_err)
    Slope_of_Maturity = Green_down / DOM
    values.append(Slope_of_Maturity)
    Slope_of_Maturity_err = multiplicative_error(Slope_of_Maturity, Green_down, Green_down_err,
                                                 DOM, DOM_err)
    errors.append(Slope_of_Maturity_err)
    #
    labels.append(f'Autumn {vi_name} range')
    AVI_range = AVI_max - VI_min
    values.append(AVI_range)
    AVI_range_err = additive_error(VI_min_err, AVI_max_err)
    errors.append(AVI_range_err)
    #
    labels.append(f'Autumn {vi_name} inflection')
    VI_AI = (VI_min + AVI_max) / 2.0
    values.append(VI_AI)
    VI_AI_err = AVI_range_err
    errors.append(VI_AI_err)
    #
    labels.append(f'Autumn {vi_name} inflection [DOY]')
    AI = (EOM + EOA) / 2.0
    values.append(AI)
    AI_err = additive_error(EOM_err, EOA_err)
    errors.append(AI_err)
    #
    labels.append(f'Autumn {vi_name} duration [d]')
    DOA = EOA - EOM
    values.append(DOA)
    DOA_err = AI_err
    errors.append(DOA_err)
    #
    labels.append(f'Autumn {vi_name} slope [1/d]')
    Slope_of_Autumn = AVI_range / DOA
    values.append(Slope_of_Autumn)
    Slope_of_Autumn_err = multiplicative_error(Slope_of_Autumn, AVI_range, AVI_range_err,
                                               DOA, DOA_err)
    errors.append(Slope_of_Autumn_err)
    #
    labels.append(f'Duration between {vi_name} inflections [d]')
    D_SI_AI = AI - SI
    values.append(D_SI_AI)
    D_SI_AI_err = additive_error(SI_err, AI_err)
    errors.append(D_SI_AI_err)
    #
    labels.append(f'Duration of {vi_name} season [d]')
    D_SOS_EOA = EOA - SOS
    values.append(D_SOS_EOA)
    D_SOS_EOA_err = additive_error(SOS_err, EOA_err)
    errors.append(D_SOS_EOA_err)
    #
    labels.append(f'Spring {vi_name} area [d]')
    AOS = (SVI_range * DOS) / 2.0
    values.append(AOS)
    AOS_err = multiplicative_error(AOS, SVI_range, SVI_range_err, DOS, DOS_err)
    errors.append(AOS_err)
    #
    labels.append(f'Maturity {vi_name} area [d]')
    avg_VI_range = (SVI_range + AVI_range) / 2.0
    avg_VI_range_err = additive_error(SVI_range_err, AVI_range_err)
    AOM = DOM * avg_VI_range
    values.append(AOM)
    AOM_err = multiplicative_error(AOM, DOM, DOM_err, avg_VI_range, avg_VI_range_err)
    errors.append(AOM_err)
    #
    labels.append(f'Autumn {vi_name} area [d]')
    AOA = (AVI_range * DOA) / 2.0
    values.append(AOA)
    AOA_err = multiplicative_error(AOA, AVI_range, AVI_range_err, DOA, DOA_err)
    errors.append(AOA_err)
    #
    labels.append(f'Total {vi_name} area [d]')
    TA = AOS + AOM + AOA
    values.append(TA)
    TA_err = additive_error(AOM_err, additive_error(AOS_err, AOA_err))
    errors.append(TA_err)
    #
    return labels, values, errors


def get_vi_fit_metrics(metavals, pheno_fit_params_df):
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    x_basis = metavals['x_basis']
    curve_str = metavals['vi_curve_str']
    analysis_begin_year = metavals['analysis_begin_year']
    analysis_end_year = metavals['analysis_end_year']
    outlier_round = metavals['outlier_round']
    n_outliers = metavals['n_outliers']
    fname_tuple = (point_id, vi_name, curve_str, x_basis, outlier_round)
    #
    # get curve metrics
    var_fit_params = pheno_fit_params_df[f'{vi_name}_by_{x_basis}']
    labels, values, errors = calc_fitted_pheno_curve_metrics(vi_name, var_fit_params)
    #
    # report curve indicators and calculated metrics
    print(f'Round {outlier_round}: {n_outliers} outliers identified')
    print()
    print(f'fitted {analysis_begin_year}-{analysis_end_year} mean {curve_str} curve metrics')
    for l, label in enumerate(labels):
        if (label[-1] == ']') and (label[-3] != '/'):
            print(f'  {label} = {values[l]:.1f} +/- {errors[l]:.1f}')
        else:
            print(f'  {label} = {values[l]:.4f} +/- {errors[l]:.4f}')
    print()
    #
    # create metrics dataframe 
    fit_metrics_df = pd.DataFrame({'label': labels, 'value_mean': values,
                                   'error_mean': errors})
    #
    # save metrics dataframe to CSV
    outfname = '%s_retained_%s_%s_%s_round_%d_fit_metrics.csv' % fname_tuple
    outfpath = f'{output_path}/{outfname}'
    fit_metrics_df.to_csv(outfpath, index=False)
    print(f'saved {outfname}')
    print()
    #
    return fit_metrics_df


#
# end part of phenology_model_vi_metrics.py

**Notebook Cell 18**: invoke function to calculate metrics for the long-term mean phenological curve

Calls function in **Notebook Cell 17**

In [ ]:
# part of model_vi_phenology.py
#


pheno_fit_metrics_df = get_vi_fit_metrics(metavals, pheno_fit_params_df)


#
# end part of model_vi_phenology.py

---

### **Exploratory Analysis of Phenological Residuals**

Various looks at the input variables, the VI residuals, and their properties:
- trends in the response (VI residual) variable within all or part of the growing season (explored as correlations with DOY)
- autocorrelation in the response (VI residual) variable
- correlations between input (WxCD anomaly) and response (VI residual) variables
- cross-correlation among input (WxCD anomaly) variables (collinearity)


**Notebook Cell 19**: exploratory analysis of VI residuals

In [ ]:
# Q: Are the VI residuals correlated with DOY through the whole growing season?
#

if metavals['graph_output'] and metavals['exploratory']:
    doys = np.array(pheno_fit_residuals_df['DOY'])
    residual = np.array(pheno_fit_residuals_df[f'{vi_name}_residual'])
    #
    min_val = np.min(doys)
    max_val = np.max(doys)
    regression_stats = regress(doys, residual)
    x_regression = np.arange(np.round(min_val, 2), np.round(max_val, 2) + 0.01, 0.01)
    y_regression = regression_stats[0] * x_regression + regression_stats[1]
    #
    plt.figure(figsize=(6, 4))
    plt.scatter(doys, residual, marker='x', c='k', s=30)
    plt.plot(x_regression, y_regression, 'b-', linewidth=2)
    plt.xlim([metavals['growing_season_start'], metavals['growing_season_end']])
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('DOY', fontsize=11)
    plt.ylabel(f'{vi_name} residual', fontsize=11)
    title = f'Full-year residuals (round {metavals["outlier_round"]}):'
    title += f' r = {regression_stats[2]:.4f}, p = {regression_stats[3]:.4f}'
    plt.title(title, fontsize=12)
    plt.tight_layout()

#
#

**Notebook Cell 20**: exploratory analysis of VI residuals

In [ ]:
# Q: Are the VI residuals correlated with DOY during the (spring) green-up phase?
#

if metavals['graph_output'] and metavals['exploratory']:
    var_fit_params = list(pheno_fit_metrics_df['value_mean'])
    #
    SOS = var_fit_params[1]
    SOM = var_fit_params[3]
    doys = np.array(pheno_fit_residuals_df['DOY'])
    residuals = np.array(pheno_fit_residuals_df[f'{vi_name}_residual'])
    #
    xvar = []
    yvar = []
    for d, doy in enumerate(doys):
        if (doy > SOS) and (doy <= SOM):
            xvar.append(doy)
            yvar.append(residuals[d])
    min_val = np.min(xvar)
    max_val = np.max(xvar)
    regression_stats = regress(xvar, yvar)
    x_regression = np.arange(np.round(min_val, 2), np.round(max_val, 2) + 0.01, 0.01)
    y_regression = regression_stats[0] * x_regression + regression_stats[1]
    #
    plt.figure(figsize=(6, 4))
    plt.scatter(xvar, yvar, marker='x', c='k', s=30)
    plt.plot(x_regression, y_regression, 'b-', linewidth=2)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('DOY', fontsize=11)
    plt.ylabel(f'{vi_name} residual', fontsize=11)
    title = f'Green-up residuals (round {metavals["outlier_round"]}):'
    title += f' r = {regression_stats[2]:.4f}, p = {regression_stats[3]:.4f}'
    plt.title(title, fontsize=12)
    plt.tight_layout()

#
#

**Notebook Cell 21**: exploratory analysis of VI residuals

In [ ]:
# Q: Are the VI residuals correlated with DOY during the (summer) mature phase?
#

if metavals['graph_output'] and metavals['exploratory']:
    var_fit_params = list(pheno_fit_metrics_df['value_mean'])
    #
    SOM = var_fit_params[3]
    EOM = var_fit_params[5]
    doys = np.array(pheno_fit_residuals_df['DOY'])
    residuals = np.array(pheno_fit_residuals_df[f'{vi_name}_residual'])
    #
    xvar = []
    yvar = []
    for d, doy in enumerate(doys):
        if (doy > SOM) and (doy < EOM):
            xvar.append(doy)
            yvar.append(residuals[d])
    min_val = np.min(xvar)
    max_val = np.max(xvar)
    regression_stats = regress(xvar, yvar)
    x_regression = np.arange(np.round(min_val, 2), np.round(max_val, 2) + 0.01, 0.01)
    y_regression = regression_stats[0] * x_regression + regression_stats[1]
    #
    plt.figure(figsize=(6, 4))
    plt.scatter(xvar, yvar, marker='x', c='k', s=30)
    plt.plot(x_regression, y_regression, 'b-', linewidth=2)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('DOY', fontsize=11)
    plt.ylabel(f'{vi_name} residual', fontsize=11)
    title = f'Maturity residuals (round {metavals["outlier_round"]}):'
    title += f' r = {regression_stats[2]:.4f}, p = {regression_stats[3]:.4f}'
    plt.title(title, fontsize=12)
    plt.tight_layout()

#
#

**Notebook Cell 22**: exploratory analysis of VI residuals

In [ ]:
# Q: Are the VI residuals correlated with DOY during the (autumn) senescence phase?
#

if metavals['graph_output'] and metavals['exploratory']:
    var_fit_params = list(pheno_fit_metrics_df['value_mean'])
    #
    EOM = var_fit_params[5]
    EOA = var_fit_params[6]
    doys = np.array(pheno_fit_residuals_df['DOY'])
    residuals = np.array(pheno_fit_residuals_df[f'{vi_name}_residual'])
    #
    xvar = []
    yvar = []
    for d, doy in enumerate(doys):
        if (doy >= EOM) and (doy < EOA):
            xvar.append(doy)
            yvar.append(residuals[d])
    min_val = np.min(xvar)
    max_val = np.max(xvar)
    regression_stats = regress(xvar, yvar)
    x_regression = np.arange(np.round(min_val, 2), np.round(max_val, 2) + 0.01, 0.01)
    y_regression = regression_stats[0] * x_regression + regression_stats[1]
    #
    plt.figure(figsize=(6, 4))
    plt.scatter(xvar, yvar, marker='x', c='k', s=30)
    plt.plot(x_regression, y_regression, 'b-', linewidth=2)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('DOY', fontsize=11)
    plt.ylabel(f'{vi_name} residual', fontsize=11)
    title = f'Senescence residuals (round {metavals["outlier_round"]}):'
    title += f' r = {regression_stats[2]:.4f}, p = {regression_stats[3]:.4f}'
    plt.title(title, fontsize=12)
    plt.tight_layout()

#
#

**Notebook Cell 23**: exploratory analysis of VI residuals

In [ ]:
# Q: Are the VI residuals correlated with the fitted-curve VI value through the whole growing season?
#

if metavals['graph_output'] and metavals['exploratory']:
    xvar = np.array(pheno_fit_residuals_df['DOY'])
    yvar = np.array(pheno_fit_residuals_df[f'{vi_name}_fitted'])
    #
    plt.figure(figsize=(6, 4))
    plt.scatter(xvar, yvar, marker='x', c='k', s=30)
    plt.xlim([metavals['growing_season_start'], metavals['growing_season_end']])
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('DOY', fontsize=11)
    plt.ylabel(f'fitted {vi_name}', fontsize=11)
    plt.tight_layout()
    #
    xvar = np.array(pheno_fit_residuals_df[f'{vi_name}_fitted'])
    yvar = np.array(pheno_fit_residuals_df[f'{vi_name}_residual'])
    min_val = np.min(xvar)
    max_val = np.max(xvar)
    regression_stats = regress(xvar, yvar)
    x_regression = np.arange(np.round(min_val, 2), np.round(max_val, 2) + 0.01, 0.01)
    y_regression = regression_stats[0] * x_regression + regression_stats[1]
    #
    plt.figure(figsize=(6, 6))
    plt.scatter(xvar, yvar, marker='x', c='k', s=30)
    plt.plot(x_regression, y_regression, 'b-', linewidth=2)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel(f'fitted {vi_name}', fontsize=11)
    plt.ylabel(f'{vi_name} residual', fontsize=11)
    title = f'Full-year residuals (round {metavals["outlier_round"]}):'
    title += f' r = {regression_stats[2]:.4f}, p = {regression_stats[3]:.4f}'
    plt.title(title, fontsize=12)
    plt.tight_layout()

#
#

**Notebook Cell 24**: exploratory analysis of VI residuals

In [ ]:
# Q: Are the VI residuals correlated with the fitted-curve VI value during the (spring) green-up phase?
#

if metavals['graph_output'] and metavals['exploratory']:
    var_fit_params = list(pheno_fit_metrics_df['value_mean'])
    #
    SOS = var_fit_params[1]
    SOM = var_fit_params[3]
    doys = np.array(pheno_fit_residuals_df['DOY'])
    residuals = np.array(pheno_fit_residuals_df[f'{vi_name}_residual'])
    fitted = np.array(pheno_fit_residuals_df[f'{vi_name}_fitted'])
    #
    xvar = []
    yvar = []
    for d, doy in enumerate(doys):
        if (doy > SOS) and (doy <= SOM):
            xvar.append(fitted[d])
            yvar.append(residuals[d])
    min_val = np.min(xvar)
    max_val = np.max(xvar)
    regression_stats = regress(xvar, yvar)
    x_regression = np.arange(np.round(min_val, 2), np.round(max_val, 2) + 0.01, 0.01)
    y_regression = regression_stats[0] * x_regression + regression_stats[1]
    #
    plt.figure(figsize=(6, 6))
    plt.scatter(xvar, yvar, marker='x', c='k', s=30)
    plt.plot(x_regression, y_regression, 'b-', linewidth=2)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel(f'fitted {vi_name}', fontsize=11)
    plt.ylabel(f'{vi_name} residual', fontsize=11)
    title = f'Green-up residuals (round {metavals["outlier_round"]}):'
    title += f' r = {regression_stats[2]:.4f}, p = {regression_stats[3]:.4f}'
    plt.title(title, fontsize=12)
    plt.tight_layout()

#
#

**Notebook Cell 25**: exploratory analysis of VI residuals

In [ ]:
# Q: Are the VI residuals correlated with the fitted-curve VI value during the (summer) mature phase?
#

if metavals['graph_output'] and metavals['exploratory']:
    var_fit_params = list(pheno_fit_metrics_df['value_mean'])
    #
    SOM = var_fit_params[3]
    EOM = var_fit_params[5]
    doys = np.array(pheno_fit_residuals_df['DOY'])
    residuals = np.array(pheno_fit_residuals_df[f'{vi_name}_residual'])
    fitted = np.array(pheno_fit_residuals_df[f'{vi_name}_fitted'])
    #
    xvar = []
    yvar = []
    for d, doy in enumerate(doys):
        if (doy > SOM) and (doy < EOM):
            xvar.append(fitted[d])
            yvar.append(residuals[d])
    min_val = np.min(xvar)
    max_val = np.max(xvar)
    regression_stats = regress(xvar, yvar)
    x_regression = np.arange(np.round(min_val, 2), np.round(max_val, 2) + 0.01, 0.01)
    y_regression = regression_stats[0] * x_regression + regression_stats[1]
    #
    plt.figure(figsize=(6, 6))
    plt.scatter(xvar, yvar, marker='x', c='k', s=30)
    plt.plot(x_regression, y_regression, 'b-', linewidth=2)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel(f'fitted {vi_name}', fontsize=11)
    plt.ylabel(f'{vi_name} residual', fontsize=11)
    title = f'Maturity residuals (round {metavals["outlier_round"]}):'
    title += f' r = {regression_stats[2]:.4f}, p = {regression_stats[3]:.4f}'
    plt.title(title, fontsize=12)
    plt.tight_layout()

#
#

**Notebook Cell 26**: exploratory analysis of VI residuals

In [ ]:
# Q: Are the VI residuals correlated with the fitted-curve VI value during the (autumn) senescence phase?
#

if metavals['graph_output'] and metavals['exploratory']:
    var_fit_params = list(pheno_fit_metrics_df['value_mean'])
    #
    EOM = var_fit_params[5]
    EOA = var_fit_params[6]
    doys = np.array(pheno_fit_residuals_df['DOY'])
    residuals = np.array(pheno_fit_residuals_df[f'{vi_name}_residual'])
    fitted = np.array(pheno_fit_residuals_df[f'{vi_name}_fitted'])
    #
    xvar = []
    yvar = []
    for d, doy in enumerate(doys):
        if (doy >= EOM) and (doy < EOA):
            xvar.append(fitted[d])
            yvar.append(residuals[d])
    min_val = np.min(xvar)
    max_val = np.max(xvar)
    regression_stats = regress(xvar, yvar)
    x_regression = np.arange(np.round(min_val, 2), np.round(max_val, 2) + 0.01, 0.01)
    y_regression = regression_stats[0] * x_regression + regression_stats[1]
    #
    plt.figure(figsize=(6, 6))
    plt.scatter(xvar, yvar, marker='x', c='k', s=30)
    plt.plot(x_regression, y_regression, 'b-', linewidth=2)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel(f'fitted {vi_name}', fontsize=11)
    plt.ylabel(f'{vi_name} residual', fontsize=11)
    title = f'Senescence residuals (round {metavals["outlier_round"]}):'
    title += f'r = {regression_stats[2]:.4f} (p = {regression_stats[3]:.4f})'
    plt.title(title, fontsize=12)
    plt.tight_layout()

#
#

**Notebook Cell 27**: exploratory analysis of VI residuals

In [ ]:
# Q: Do the VI residuals show any patterns through the year that are common across all years?
#

if metavals['graph_output'] and metavals['exploratory']:
    dates = np.array(pheno_fit_residuals_df['DYear'])
    doys = np.array(pheno_fit_residuals_df['DOY'])
    min_year = int(dates[0])
    max_year = int(dates[-1]) + 1
    residuals = np.array(pheno_fit_residuals_df[f'{vi_name}_residual'])
    #
    plt.figure(figsize=(6, 4))
    for year in range(min_year, max_year+1):
        xvar = []
        yvar = []
        for d, dyear in enumerate(dates):
            if int(dyear) == year:
                xvar.append(doys[d])
                yvar.append(residuals[d])
        plt.plot(xvar, yvar, 'b-', marker='o', linewidth=1)
    plt.xlim([metavals['growing_season_start'], metavals['growing_season_end']])
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('DOY', fontsize=11)
    plt.ylabel(f'{vi_name} residual', fontsize=11)
    plt.tight_layout()

#
#

**Notebook Cell 28**: exploratory analysis of VI residuals

In [ ]:
# Q: Do the VI residuals in each year show any pattern across all years?
#

if metavals['graph_output'] and metavals['exploratory']:
    dates = np.array(pheno_fit_residuals_df['DYear'])
    min_year = int(dates[0])
    max_year = int(dates[-1]) + 1
    residuals = np.array(pheno_fit_residuals_df[f'{vi_name}_residual'])
    #
    # detrend and fit a sine curve here 
    #
    plt.figure(figsize=(12, 4))
    for year in range(min_year, max_year+1):
        xvar = []
        yvar = []
        for d, dyear in enumerate(dates):
            if int(dyear) == year:
                xvar.append(dyear)
                yvar.append(residuals[d])
        plt.plot(xvar, yvar, 'b-', marker='o', linewidth=1)
    # add fitted sine curve
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('Year', fontsize=11)
    plt.ylabel(f'{vi_name} residual', fontsize=11)
    plt.tight_layout()

#
#

**Notebook Cell 29**: exploratory analysis of the long-term mean phenological curve

In [ ]:
# Q: Is the VI fitted curve autocorrelated at any particular lag?
#

if metavals['graph_output'] and metavals['exploratory']:
    coeff = []
    lag_range = range(1, 31)
    for lag in lag_range:
        stats = regress(full_year_mean_vi[:-lag], full_year_mean_vi[lag:])
        coeff.append(stats[2])
    #
    plt.figure(figsize=(6, 4))
    plt.plot(lag_range, coeff, 'b-', marker='o', linewidth=2)
    plt.xlabel('lag [d]', fontsize=11)
    plt.ylabel('r', fontsize=11)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()

#
#

**Notebook Cell 30**: exploratory analysis of VI residuals and the long-term mean phenological curve

In [ ]:
# Q: Are the VI residuals correlated with the fitted curve at any particular lag?
#

if metavals['graph_output'] and metavals['exploratory']:
    residual = np.array(pheno_fit_residuals_df[f'{vi_name}_residual'])
    coeff = []
    pval = []
    lag_range = range(91)
    for lag in lag_range:
        doys = np.array(pheno_fit_residuals_df['DOY'])
        lagged_doys = doys - lag
        lagged_fitted = [full_year_mean_vi[d-1] for d in lagged_doys]
        stats = regress(residual, lagged_fitted)
        coeff.append(stats[2])
        pval.append(stats[3])
    #
    plt.figure(figsize=(6, 4))
    plt.plot(lag_range, coeff, 'b-', marker='o', linewidth=2)
    plt.xlabel(f'lagged fitted {vi_name} [d]', fontsize=11)
    plt.ylabel('r', fontsize=11)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()
    #
    plt.figure(figsize=(6, 4))
    plt.plot(lag_range, pval, 'b-', marker='o', linewidth=2)
    plt.xlabel(f'lagged fitted {vi_name} [d]', fontsize=11)
    plt.ylabel('p', fontsize=11)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.tight_layout()

#
#

**Notebook Cell 31**: exploratory analysis of correlations between VI residuals and WxCD anomalies

In [ ]:
# Q: Are any particular WxCD anomalies correlated with the VI residuals?
#

if metavals['graph_output'] and metavals['exploratory']:
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    x_basis = metavals['x_basis']
    curve_str = metavals['vi_curve_str']
    outlier_round = metavals['outlier_round']
    fname_tuple = (point_id, vi_name, curve_str, x_basis, outlier_round)
    #
    slope = []
    intercept = []
    correlation = []
    significance = []
    marker = []
    #
    # get VI residuals
    residual = np.array(pheno_fit_residuals_df[f'{vi_name}_residual'])
    #
    for v, var in enumerate(wxcd_std_anom_vars):
        wxcd_stnd_anom = np.array(wxcd_vi_df[var])
        regression_stats = regress(wxcd_stnd_anom, residual)
        slope.append(regression_stats[0])
        intercept.append(regression_stats[1])
        correlation.append(regression_stats[2])
        significance.append(regression_stats[3])
        if regression_stats[3] < 0.000001:
            marker.append('******')
        elif regression_stats[3] < 0.00001:
            marker.append('*****')
        elif regression_stats[3] < 0.0001:
            marker.append('****')
        elif regression_stats[3] < 0.001:
            marker.append('***')
        elif regression_stats[3] < 0.01:
            marker.append('**')
        elif regression_stats[3] < 0.05:
            marker.append('*')
        else:
            marker.append(' ')
    #
    wxcd_corr_df = pd.DataFrame({'variable': wxcd_std_anom_vars, 'marker': marker,
                                 'slope': slope, 'intercept': intercept,
                                 'corr': correlation, 'sig': significance})
    print(f'Round {outlier_round} {vi_name} residuals vs. WxCD standardized anomalies')
    wxcd_corr_df.sort_values(by='sig', ascending=True, inplace=True)
    wxcd_corr_df = wxcd_corr_df.reset_index(drop=True)
    print(wxcd_corr_df)
    print()
    #
    outfname = '%s_retained_%s_%s_%s_round_%d_residual_WxCD_regressions.csv' % fname_tuple
    outfpath = f'{output_path}/{outfname}'
    wxcd_corr_df.to_csv(outfpath)
    print(f'saved {outfname}')
    print()
    #
    # get WxCD anomaly values and regress/plot against residuals
    plot_width = 4
    plot_height = 3
    fig_ncols = 3
    fig_width = plot_width * fig_ncols
    fig_nrows = int(np.ceil(len(wxcd_std_anom_vars) / fig_ncols))
    fig_height = plot_height * fig_nrows
    plt.figure(figsize=(fig_width, fig_height))
    #
    wxcd_vars_ordered = list(wxcd_corr_df['variable'])
    for v, var in enumerate(wxcd_vars_ordered):
        position = v + 1
        # print(f'{position}', end=' ')
        wxcd_stnd_anom = np.array(wxcd_vi_df[var])
        min_val = np.min(wxcd_stnd_anom)
        max_val = np.max(wxcd_stnd_anom)
        regression_stats = regress(wxcd_stnd_anom, residual)
        x_regression = np.arange(np.round(min_val, 2), np.round(max_val, 2) + 0.01, 0.01)
        y_regression = regression_stats[0] * x_regression + regression_stats[1]
        #
        if regression_stats[3] <= 0.05:
            plt.subplot(fig_nrows, fig_ncols, position, facecolor='lightgray')
            plt.scatter(wxcd_stnd_anom, residual, marker='x', c='k', s=30)
            plt.plot(x_regression, y_regression, 'r-', linewidth=2)
        else:
            plt.subplot(fig_nrows, fig_ncols, position)
            plt.scatter(wxcd_stnd_anom, residual, marker='x', c='k', s=30)
            plt.plot(x_regression, y_regression, 'b-', linewidth=2)
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        plt.xlabel(var, fontsize=11)
        plt.ylabel(f'{vi_name} residual', fontsize=11)
        plt.title(f'r = {regression_stats[2]:.4f}, p = {regression_stats[3]:.4f}')
        plt.tight_layout()
    #
    plotfname = '%s_retained_%s_%s_%s_round_%d_residual_WxCD_regressions.png' % fname_tuple
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    print()

#
#

**Notebook Cell 32**: exploratory analysis of cross-correlations among WxCD anomalies

In [ ]:
# Q: Are any particular WxCD anomalies highly cross-correlated?
#

if metavals['graph_output'] and metavals['exploratory']:
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    #
    wxcd_stnd_anom_df = wxcd_vi_df.loc[:, wxcd_std_anom_vars]
    wxcd_stnd_anom_corr_df = wxcd_stnd_anom_df.corr()
    #
    plt.figure(figsize=(12, 12))
    heatmap = sns.heatmap(wxcd_stnd_anom_corr_df, vmin=-1, vmax=1, annot=False, cmap='BrBG')
    # heatmap.set_title(f'WxCD Cross-correlations at {point_id}', fontdict={'fontsize':12}, pad=12)
    #
    plotfname = f'{point_id}_all_WxCD_heatmap.png'
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    print()


**Notebook Cell 33**: plot selected WxCD values and anomalies

In [ ]:
# not yet part of phenology_model_plots.py
#


def plot_wxcd_var_by_dyear(var_obs_name, units, var_dyears, var_obs_values,
                           var_std_anom_name, var_std_anom_values,
                           begin_year, end_year, output_path, point_id):
    #
    fig = plt.figure(figsize=(12,8))
    #
    # plot WxCD values on Landsat dates by DYear
    ax = plt.subplot(2, 1, 1)
    plt.scatter(var_dyears, var_obs_values, marker='x', c='k', s=30)
    plt.xlim([begin_year, end_year+1])
    years = np.arange(begin_year, end_year+2)
    even_years = [year if year % 2 == 0 else '' for year in years]
    plt.xticks(years, fontsize=10)
    ax.set_xticklabels(even_years, fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('Year', fontsize=11)
    plt.ylabel(f'{var_obs_name} [{units}]', fontsize=11)
    title = f'{begin_year}-{end_year} {var_obs_name} observations'
    plt.title(title, fontsize=12)
    #
    # plot WxCD anomalies on Landsat dates by DYear
    ax = plt.subplot(2, 1, 2)
    plt.scatter(var_dyears, var_std_anom_values, marker='x', c='k', s=30)
    plt.plot([begin_year, end_year+1], [0.0, 0.0], 'b-', linewidth=2)
    plt.xlim([begin_year, end_year+1])
    years = np.arange(begin_year, end_year+2)
    even_years = [year if year % 2 == 0 else '' for year in years]
    plt.xticks(years, fontsize=10)
    ax.set_xticklabels(even_years, fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('Year', fontsize=11)
    plt.ylabel(var_std_anom_name, fontsize=11)
    title = f'{begin_year}-{end_year} {var_obs_name} standardized anomalies'
    plt.title(title, fontsize=12)
    #
    plt.tight_layout()
    plotfname = f'{point_id}_{var_obs_name}_observations+std_anoms_by_dyear.png'
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    #
    return


**Notebook Cell 34**: plot three specific WxCD variables (values and anomalies) as examples

Calls function in **Notebook Cell 33**

In [ ]:
# Show some specific WxCD observations and climatological anomalies
#

if metavals['graph_output'] and metavals['exploratory']:
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    begin_year = metavals['analysis_begin_year']
    end_year = metavals['analysis_end_year']
    #
    # get and plot a significantly correlated prcp variable
    var_obs_name = 'prcp_365d_sum'
    var_std_anom_name = 'prcp_365d_std_anom'
    var_dyears = np.array(wxcd_vi_df['DYear'])
    var_obs_values = np.array(wxcd_vi_df[var_obs_name])
    var_std_anom_values = np.array(wxcd_vi_df[var_std_anom_name])
    plot_wxcd_var_by_dyear(var_obs_name, 'mm', var_dyears, var_obs_values,
                           var_std_anom_name, var_std_anom_values,
                           begin_year, end_year, output_path, point_id)
    #
    # get and plot a significantly correlated tmin/tmax variable
    var_obs_name = 'tmin_45d_avg'
    var_std_anom_name = 'tmin_45d_std_anom'
    var_dyears = np.array(wxcd_vi_df['DYear'])
    var_obs_values = np.array(wxcd_vi_df[var_obs_name])
    var_std_anom_values = np.array(wxcd_vi_df[var_std_anom_name])
    plot_wxcd_var_by_dyear(var_obs_name, r'$^{\circ}$C', var_dyears, var_obs_values,
                           var_std_anom_name, var_std_anom_values,
                           begin_year, end_year, output_path, point_id)
    #
    # get and plot an uncorrelated variable
    var_obs_name = 'tavg_10d_avg'
    var_std_anom_name = 'tavg_10d_std_anom'
    var_dyears = np.array(wxcd_vi_df['DYear'])
    var_obs_values = np.array(wxcd_vi_df[var_obs_name])
    var_std_anom_values = np.array(wxcd_vi_df[var_std_anom_name])
    plot_wxcd_var_by_dyear(var_obs_name, r'$^{\circ}$C', var_dyears, var_obs_values,
                           var_std_anom_name, var_std_anom_values,
                           begin_year, end_year, output_path, point_id)


---

### **PLS Regression on Phenological Residuals**

*Description and details here!*

**Notebook Cell 35**: statistical calculations to de-trend and re-trend a time series variable

In [ ]:
# part of phenology_model_stats.py
#


def detrend_given_stats(x, y, r_params):
    slope, intercept = r_params[:2]
    detrended_y = y - (slope * x + intercept)
    #
    return detrended_y


def detrend(x, y):
    r_params = regress(x, y)
    detrended_y = detrend_given_stats(x, y, r_params)
    #
    return r_params, detrended_y


def retrend(x, detrended_y, r_params):
    slope, intercept = r_params[:2]
    y = detrended_y + (slope * x + intercept)
    #
    return y


#
# end part of phenology_model_stats.py

**Notebook Cell 36**: statistical calculations to standardize (center and scale) and de-standardize a time series variable

In [ ]:
# part of phenology_model_stats.py
#


def standardize_given_stats(y, y_stats):
    y_mean, y_std = y_stats
    if y_std:
        standardized_y = (y - y_mean) / y_std
    else:
        standardized_y = np.zeros_like(y)
    #
    return standardized_y


def standardize(y):
    y_mean = np.mean(y)
    y_std = np.std(y)
    standardized_y = standardize_given_stats(y, [y_mean, y_std])
    #
    return [y_mean, y_std], standardized_y


def destandardize(standardized_y, y_stats):
    y_mean, y_std = y_stats
    y = (standardized_y * y_std) + y_mean
    #
    return y


#
# end part of phenology_model_stats.py

**Notebook Cell 37**: prepare input VI and WxCD variables (time series) for PLS regression

Calls functions in **Notebook Cells 35 and 36**

In [ ]:
# contents of phenology_model_plsr_prep.py
#


def prep_vars(vi_name, wxcd_vars, wxcd_vi_df, pheno_fit_anoms_df):
    #
    # set up output data structures
    trend_params_cols = ['parameter', f'{vi_name}_fitted', f'{vi_name}_residual']
    for var in wxcd_vars:
        trend_params_cols.append(var)
    trend_params_df = pd.DataFrame(columns=trend_params_cols)
    trend_params_df['parameter'] = ['trend_slope', 'trend_intercept',
                                    'trend_corr', 'trend_sig', 'trend_stderr']
    #
    stnd_params_cols = ['parameter', f'{vi_name}_fitted_detrended',
                        f'{vi_name}_residual_detrended']
    for var in wxcd_vars:
        stnd_params_cols.append(f'{var}_detrended')
    stnd_params_df = pd.DataFrame(columns=stnd_params_cols)
    stnd_params_df['parameter'] = ['mean', 'std']
    #
    # detrend and standardize fitted VI values
    dyear = np.array(pheno_fit_anoms_df['DYear'])
    vi_fitted = np.array(pheno_fit_anoms_df[f'{vi_name}_fitted'])
    detrend_params, vi_fitted_detrended = detrend(dyear, vi_fitted)
    trend_params_df[f'{vi_name}_fitted'] = detrend_params
    pheno_fit_anoms_df[f'{vi_name}_fitted_detrended'] = vi_fitted_detrended
    stnd_params, vi_fitted_detrended_stnd = standardize(vi_fitted_detrended)
    stnd_params_df[f'{vi_name}_fitted_detrended'] = stnd_params
    pheno_fit_anoms_df[f'{vi_name}_fitted_detrended_stnd'] = vi_fitted_detrended_stnd
    #
    # detrend and standardize residual VI values
    vi_residual = np.array(pheno_fit_anoms_df[f'{vi_name}_residual'])
    detrend_params, vi_residual_detrended = detrend(dyear, vi_residual)
    trend_params_df[f'{vi_name}_residual'] = detrend_params
    pheno_fit_anoms_df[f'{vi_name}_residual_detrended'] = vi_residual_detrended
    stnd_params, vi_residual_detrended_stnd = standardize(vi_residual_detrended)
    stnd_params_df[f'{vi_name}_residual_detrended'] = stnd_params
    pheno_fit_anoms_df[f'{vi_name}_residual_detrended_stnd'] = vi_residual_detrended_stnd
    #
    # obtain/calculate WxCD anomalies for retained VI dates
    clim_stnd_anoms_cols = deepcopy(wxcd_vars)
    for var in wxcd_vars:
        clim_stnd_anoms_cols.append(f'{var}_detrended')
        clim_stnd_anoms_cols.append(f'{var}_detrended_stnd')
    #
    clim_stnd_anoms_df = pd.DataFrame(columns=clim_stnd_anoms_cols)
    for var in wxcd_vars:
        clim_stnd_anoms_df[var] = np.array(wxcd_vi_df[var])
    #
    # detrend and standardize WxCD anomaly values
    for var in wxcd_vars:
        wxcd_stnd_anom = np.array(clim_stnd_anoms_df[var])
        detrend_params, wxcd_stnd_anom_detrended = detrend(dyear, wxcd_stnd_anom)
        trend_params_df[var] = detrend_params
        clim_stnd_anoms_df[f'{var}_detrended'] = wxcd_stnd_anom_detrended
        stnd_params, wxcd_stnd_anom_detrended_stnd = standardize(wxcd_stnd_anom_detrended)
        stnd_params_df[f'{var}_detrended'] = stnd_params
        clim_stnd_anoms_df[f'{var}_detrended_stnd'] = wxcd_stnd_anom_detrended_stnd
    #
    # concatenate the VI (pheno_fit_anoms) and WxCD (clim_stnd_anoms) dataframes
    pheno_fit_anoms_df = pd.concat([pheno_fit_anoms_df, clim_stnd_anoms_df], axis=1)
    #
    return pheno_fit_anoms_df, trend_params_df, stnd_params_df


def plsr_prep(metavals, wxcd_vars, wxcd_vi_retained_df, pheno_fit_anoms_df):
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    curve_str = metavals['vi_curve_str']
    x_basis = metavals['x_basis']
    outlier_round = metavals['outlier_round']
    fname_tuple = (point_id, vi_name, curve_str, x_basis, outlier_round)
    #
    print(f'Preparing round {outlier_round} {vi_name} residuals and WxCD for PLS regression')
    print()
    #
    pheno_fit_anoms_df, trend_params_df, stnd_params_df = \
        prep_vars(vi_name, wxcd_vars, wxcd_vi_retained_df, pheno_fit_anoms_df)
    #
    # save retained detrended and standardized VI and WxCD anomalies
    outfname = '%s_pls_retained_%s_%s_%s_round_%d_fit_wxcd_stnd_anoms.csv' % fname_tuple
    outfpath = f'{output_path}/{outfname}'
    pheno_fit_anoms_df.to_csv(outfpath, index=False)
    print(f'saved {outfname}')
    #
    # save trend parameters for all vars (VI and WxCD)
    outfname = '%s_pls_retained_%s_%s_%s_round_%d_trend_params.csv' % fname_tuple
    outfpath = f'{output_path}/{outfname}'
    trend_params_df.to_csv(outfpath, index=False)
    print(f'saved {outfname}')
    #
    # save standardization parameters for all vars (VI and WxCD)
    outfname = '%s_pls_retained_%s_%s_%s_round_%d_stnd_params.csv' % fname_tuple
    outfpath = f'{output_path}/{outfname}'
    stnd_params_df.to_csv(outfpath, index=False)
    print(f'saved {outfname}')
    print()
    #
    return trend_params_df, stnd_params_df, pheno_fit_anoms_df


#
# end contents of phenology_model_plsr_prep.py

**Notebook Cell 38**: invoke variable preparation prior to PLS regression

Calls function in **Notebook Cell 37**

In [ ]:
# part of model_vi_phenology.py
#


print('PLS Regression for Parameter Reduction')
print()
#
# setup output data structure
plsr_results = {'point_id': metavals['point_id'],
                'vi_name': metavals['vi_name'],
                'curve_str': metavals['vi_curve_str'],
                'x_basis': metavals['x_basis'],
                'vip_threshold': metavals['vip_threshold']}
#
# prepare variables
trend_params_df, stnd_params_df, pheno_fit_residuals_df = \
    plsr_prep(metavals, wxcd_std_anom_vars, wxcd_vi_df, pheno_fit_residuals_orig_df)


#
# end part of model_vi_phenology.py

---

**Notebook Cell 39**: actual PLS regression and prediction, with VIP calculation

In [ ]:
# part of phenology_model_plsr.py
#
# many thanks to Townsend Lab member Adam Chlus for the VIP calculation
#
# optimizing this one function for GPU would be IDEAL
#


def pls_prediction(x_train, y_train, x_test, n_components):
    """
    Partial least-squares (PLS) regression and prediction process
    - regress, predict, extract coefficients, and calculate VIP
    
    Help with SVD convergence error handling from
    http://stackoverflow.com/questions/9155478/
    """
    pls_model = PLSRegression(n_components=n_components)
    try:
        pls_model.fit(x_train, y_train)
    except np.linalg.linalg.LinAlgError as err:
        if 'SVD did not converge' in err.print:
            raise ValueError('ERROR: PLS SVD did not converge')
    y_train_pred = pls_model.predict(x_train)
    y_test_pred = pls_model.predict(x_test)
    #
    # gather model coefficients and intercept
    coeffs_list = pls_model.coef_[0]
    coeffs_arr = np.array(coeffs_list)
    intercept = pls_model.intercept_[0]
    #
    # calculate VIP statistics
    vip_list = []
    nvars = len(coeffs_list)
    loadings = pls_model.y_loadings_
    scores = pls_model.x_scores_
    sum_squares = loadings**2 + scores**2
    weights = pls_model.x_weights_
    sum_weights = np.sum(weights**2, axis=0)
    for i in range(nvars):      
        var_importance = sum_squares * (weights[i]**2) / sum_weights 
        vip_val = np.sqrt(nvars * var_importance.sum() / sum_squares.sum())
        vip_list.append(vip_val)
    vip_arr = np.array(vip_list)
    #
    return y_train_pred, y_test_pred, coeffs_arr, intercept, vip_arr


#
# end part of phenology_model_plsr.py

**Notebook Cell 40**: plot error metrics for multiple iterations of the PLS regression and prediction procedure over component space

In [ ]:
# contents of phenology_model_plsr_plots.py
#


def plot_error_metric(vi_name, nc_min, nc, metric_arr, metric_name,
                      outlier_round, passnum, nc_best=0):
    nc_range = range(nc_min, nc_min + nc)
    plt.plot(nc_range, metric_arr, 'b-', marker='o', linewidth=2)
    if np.max(metric_arr) > 0.0 and np.min(metric_arr) < 0.0:
        plt.plot(nc_range, np.zeros_like(nc_range), 'k--', linewidth=1)
    plt.xlabel('PLS regression components', fontsize=11)
    plt.ylabel(metric_name, fontsize=11)
    if nc_range[-1] > 20:
        plt.xticks(nc_range[::2], fontsize=10)
    else:
        plt.xticks(nc_range, fontsize=10)
    plt.yticks(fontsize=10)
    title = f'{vi_name} PLSR {metric_name} (round {outlier_round}, '
    if nc_best:
        title += f'pass {passnum}, best nc = {nc_best})'
    else:
        title += f'pass {passnum})'
    plt.title(title, fontsize=12)
    plt.tight_layout()  
    #
    return


def plot_plsr_error_metrics(metavals, n_components_min, n_components_max,
                            n_components_best, rsq, press, aicc, bias, mae, rmse,
                            passnum):
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    x_basis = metavals['x_basis']
    curve_str = metavals['vi_curve_str']
    outlier_round = metavals['outlier_round']
    #
    # build figure
    plt.figure(figsize=(12, 12))
    #
    # plot AICc vs n_components
    plt.subplot(3, 2, 1)
    plot_error_metric(vi_name, n_components_min, n_components_max, aicc, 'AICc',
                      outlier_round, passnum, n_components_best)
    #
    # plot Rsq vs n_components
    plt.subplot(3, 2, 2)
    plot_error_metric(vi_name, n_components_min, n_components_max, rsq, 'Rsq',
                      outlier_round, passnum)
    #
    # plot PRESS vs n_components
    plt.subplot(3, 2, 3)
    plot_error_metric(vi_name, n_components_min, n_components_max, press, 'PRESS',
                      outlier_round, passnum)
    #
    # plot Bias vs n_components
    plt.subplot(3, 2, 4)
    plot_error_metric(vi_name, n_components_min, n_components_max, bias, 'Bias',
                      outlier_round, passnum)
    #
    # plot MAE vs n_components
    plt.subplot(3, 2, 5)
    plot_error_metric(vi_name, n_components_min, n_components_max, mae, 'MAE',
                      outlier_round, passnum)
    #
    # plot RMSE vs n_components
    plt.subplot(3, 2, 6)
    plot_error_metric(vi_name, n_components_min, n_components_max, rmse, 'RMSE',
                      outlier_round, passnum)
    #
    fname_tuple = (point_id, vi_name, curve_str, x_basis, outlier_round)
    plotfname = '%s_pls_retained_%s_%s_%s_round_%d_nc_metrics' % fname_tuple
    plotfname += f'_pass_{passnum}.png'
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    print()
    #
    return


#
# end contents of phenology_model_plsr_plots.py

**Notebook Cell 41**: iterate the PLS regression and prediction procedure to find the optimum number of model components

Calls functions in **Notebook Cells 7, 39, and 40**

In [ ]:
# part of phenology_model_plsr.py
#


def n_components_iteration(idx, x, y, n_components, test_split):
    #
    # Uses train_test_split function from sklearn.model_selection
    #
    n_obs, n_params = np.shape(x)
    #
    # split input data to train and test parts
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_split)
    #
    # run PLS regression with specified number of model components
    y_train_pred, y_test_pred, coeffs, intercept, vip = \
        pls_prediction(x_train, y_train, x_test, n_components)
    #
    # calculate error metrics from PLS test result
    y_res = y_test_pred - y_test
    fit_error_metrics = calc_fit_error_metrics(n_obs, n_params+1, y_test, y_res)
    # fit_error_metrics = [bias, mae, rmse, sse, rsq, aic, aicc, bic]
    #
    return fit_error_metrics


def find_n_components(metavals, x, y, passnum):
    """
    Find optimum number of model components (n_components at lowest AICc)
        using cross-validation with given data train/test split fraction
    """
    #
    # pull needed metavals
    vi_name = metavals['vi_name']
    nc_max = metavals['nc_max']
    test_split = metavals['test_split']
    n_iterations = metavals['n_iterations']
    graph_output = metavals['graph_output']
    #
    print(f'Checking PLS component space for optimal {vi_name} regression model')
    n_obs, n_params = np.shape(x)
    print(f'  n_parameters = {n_params}')
    n_obs_test = int(np.round(n_obs * test_split))
    n_obs_train = n_obs - n_obs_test
    print(f'  n_observations = {n_obs}')
    print(f'    n_training_obs = {n_obs_train}')
    print(f'    n_testing_obs = {n_obs_test}')
    n_components_min = 1
    n_components_max = min([n_params - 1, nc_max])
    print(f'  max n_components = {n_components_max}')
    n_components_range = range(n_components_min, n_components_max + 1)
    #
    bias_arr = np.zeros((n_iterations, len(n_components_range)))
    mae_arr = np.zeros_like(bias_arr)
    rmse_arr = np.zeros_like(bias_arr)
    press_arr = np.zeros_like(bias_arr)
    rsq_arr = np.zeros_like(bias_arr)
    aic_arr = np.zeros_like(bias_arr)
    aicc_arr = np.zeros_like(bias_arr)
    bic_arr = np.zeros_like(bias_arr)
    #
    # loop through n_components
    print('  n_components')
    for n_components in n_components_range:
        print(f'    {n_components}')
        for idx in range(n_iterations):
            fit_error_metrics = n_components_iteration(idx, x, y, n_components, test_split)
            # fit_error_metrics = [bias, mae, rmse, sse, rsq, aic, aicc, bic]
            nc = n_components - n_components_min
            bias_arr[idx, nc] = fit_error_metrics[0]
            mae_arr[idx, nc] = fit_error_metrics[1]
            rmse_arr[idx, nc] = fit_error_metrics[2]
            press_arr[idx, nc] = fit_error_metrics[3]
            rsq_arr[idx, nc] = fit_error_metrics[4]
            aic_arr[idx, nc] = fit_error_metrics[5]
            aicc_arr[idx, nc] = fit_error_metrics[6]
            bic_arr[idx, nc] = fit_error_metrics[7]
    #
    print('Results')
    rsq_mean = np.mean(rsq_arr[:, :], axis=0)
    n_components_best = n_components_min + np.argmax(rsq_mean)
    print(f'  max bootstrap Rsq = {np.max(rsq_mean):.3f} with n_components = {n_components_best}')
    #
    press_mean = np.mean(press_arr[:, :], axis=0)
    n_components_best = n_components_min + np.argmin(press_mean)
    print(f'  min bootstrap PRESS = {np.min(press_mean):.1f} with n_components = {n_components_best}')
    #
    aicc_mean = np.mean(aicc_arr[:, :], axis=0)
    n_components_best = n_components_min + np.argmin(aicc_mean)
    print(f'  min bootstrap AICc = {np.min(aicc_mean):.1f} with n_components = {n_components_best}')
    print(f'  n_components = {n_components_best}')
    #
    bias_mean = np.mean(bias_arr[:, :], axis=0)
    mae_mean = np.mean(mae_arr[:, :], axis=0)
    rmse_mean = np.mean(rmse_arr[:, :], axis=0)
    #
    if graph_output:
        plot_plsr_error_metrics(metavals, n_components_min, n_components_max, n_components_best,
                                rsq_mean, press_mean, aicc_mean, bias_mean, mae_mean, rmse_mean,
                                passnum)
    #
    return n_components_best


#
# end part of phenology_model_plsr.py

**Notebook Cell 42**: invoke procedure to find the optimum number of PLS model components and get the corresponding model details

Calls functions in **Notebook Cells 7, 39, and 41**

In [ ]:
# part of phenology_model_plsr.py
#


def pls_regression(metavals, x, y, passnum, n_components=0, print_stats=False):
    """
    Run PLS with specified number of model components and report stats.
    If no n_components specified, find optimum value by cross-validation.
    """
    #
    # pull needed metavals
    vi_name = metavals['vi_name']
    outlier_round = metavals['outlier_round']
    #
    n_obs, n_params = np.shape(x)
    #
    # if n_components not specified, find optimal value by cross-validation
    if n_components == 0:
        n_components = find_n_components(metavals, x, y, passnum)
    #
    # PLS regression with identified number of model components
    _, y_pred, coeffs, intercept, vip = \
        pls_prediction(x, y, x, n_components=n_components)
    #
    # process PLS regression results and error metrics
    y_res = y_pred - y
    fit_error_metrics = calc_fit_error_metrics(n_obs, n_params+1, y, y_res)
    # fit_error_metrics = [bias, mae, rmse, sse, rsq, aic, aicc, bic]
    #
    if print_stats:
        print(f'Round {outlier_round}: PLS regression on {vi_name} residuals')
        print(f'  n_observations = {n_obs}')
        print(f'  n_parameters = {n_params}')
        print(f'  n_components = {n_components}')
        print(f'  Bias = {fit_error_metrics[0]:.3f}')
        print(f'  MAE = {fit_error_metrics[1]:.3f}')
        print(f'  RMSE = {fit_error_metrics[2]:.3f}')
        print(f'  PRESS = {fit_error_metrics[3]:.1f}')
        print(f'  Rsq = {fit_error_metrics[4]:.3f}')
        print(f'  AIC = {fit_error_metrics[5]:.1f}')
        print(f'  AICc = {fit_error_metrics[6]:.1f}')
        print(f'  BIC = {fit_error_metrics[7]:.1f}')
    #
    return n_components, fit_error_metrics, coeffs, intercept, vip


#
# end part of phenology_model_plsr.py

**Notebook Cell 43**: invoke the procedure to build and optimize a PLS model, and identify the variables with VIP ≥ 1.0

Calls function in **Notebook Cell 42**

In [ ]:
# part of phenology_model_plsr.py
# 


def vip_threshold_plsr(metavals, x, y, variables, passnum, n_components=0,
                       vip_threshold=1.0, print_stats=False):
    """
    VIP-based selection of the most important parameters (variables).
    """
    print(f'Selecting parameters using VIP threshold = {vip_threshold:.2f}')
    n_components, fit_error_metrics, betas, intercept, vip = \
        pls_regression(metavals, x, y, passnum, n_components=n_components,
                       print_stats=print_stats)
    top_var_idxs = []
    best_vars = []
    best_vip = []
    best_beta = []
    for k, var in enumerate(variables):
        if abs(vip[k]) > vip_threshold:
            top_var_idxs.append(k)
            best_vars.append(var)
            best_vip.append(vip[k])
            best_beta.append(betas[k])
    if vip_threshold > 0.0:
        print(f'- VIP threshold includes {len(best_vars)} of {len(variables)} parameters')
    vip_df = pd.DataFrame({'varname': best_vars, 'vip': best_vip, 'beta': best_beta})
    print()
    #
    return vip_df, n_components, fit_error_metrics, betas, intercept, vip


#
# end part of phenology_model_plsr.py

**Notebook Cell 44**: compile information on the PLS model produced at a given step in the iterative process loop

In [ ]:
# part of phenology_model_plsr.py
# 


def compile_step_results(plsr_results, step, vi_name, n_obs, outlier_round,
                         n_outliers, wxcd_std_anom_vars, n_components, vip,
                         betas, intercept, fit_error_metrics):
    var_prefix = f'{vi_name}_round_{outlier_round}_{step}'
    plsr_results[f'{var_prefix}_n_obs'] = n_obs
    plsr_results[f'{var_prefix}_n_outliers'] = n_outliers
    plsr_results[f'{var_prefix}_vars'] = wxcd_std_anom_vars
    plsr_results[f'{var_prefix}_nvars'] = len(wxcd_std_anom_vars)
    plsr_results[f'{var_prefix}_n_components'] = n_components
    plsr_results[f'{var_prefix}_vip'] = vip
    plsr_results[f'{var_prefix}_betas'] = betas
    plsr_results[f'{var_prefix}_intercept'] = intercept
    plsr_results[f'{var_prefix}_bias'] = fit_error_metrics[0]
    plsr_results[f'{var_prefix}_mae'] = fit_error_metrics[1]
    plsr_results[f'{var_prefix}_rmse'] = fit_error_metrics[2]
    plsr_results[f'{var_prefix}_press'] = fit_error_metrics[3]
    plsr_results[f'{var_prefix}_rsq'] = fit_error_metrics[4]
    plsr_results[f'{var_prefix}_aic'] = fit_error_metrics[5]
    plsr_results[f'{var_prefix}_aicc'] = fit_error_metrics[6]
    plsr_results[f'{var_prefix}_bic'] = fit_error_metrics[7]    
    #
    return plsr_results


#
# end part of phenology_model_plsr.py

---

**Notebook Cell 45**: Step 1 in the iterative PLS modeling loop, using all available WxCD variables

Calls functions in **Notebook Cells 42, 43, and 44**

In [ ]:
# contents of phenology_model_plsr_step_1.py
#


def get_wxcd_detrended_stnd(vi_name, wxcd_vars, pheno_fit_residuals_df):
    dyear = np.array(pheno_fit_residuals_df['DYear'])
    wxcd_vars = [v for v in wxcd_vars if vi_name not in v]
    wxcd_detrended_stnd = np.zeros((len(dyear), len(wxcd_vars)+1))
    for n, var in enumerate(wxcd_vars):
        var_arr = np.array(pheno_fit_residuals_df[f'{var}_detrended_stnd'])
        if var_arr.ndim > 1:
            wxcd_detrended_stnd[:, n] = var_arr[:, 0]
        else:
            wxcd_detrended_stnd[:, n] = var_arr
    wxcd_detrended_stnd[:, len(wxcd_vars)] = \
        np.array(pheno_fit_residuals_df[f'{vi_name}_fitted_detrended_stnd'])
    wxcd_vars.append(f'{vi_name}_fitted')
    #
    return wxcd_detrended_stnd


def get_vi_detrended_stnd(vi_name, pheno_fit_residuals_df):
    vi_detrended_stnd = \
        np.array(pheno_fit_residuals_df[f'{vi_name}_residual_detrended_stnd'])
    #
    return vi_detrended_stnd


def plsr_step_1(metavals, plsr_results, wxcd_std_anom_vars,
                wxcd_detrended_stnd, vi_detrended_stnd):
    #
    # pull needed metavals
    vi_name = metavals['vi_name']
    outlier_round = metavals['outlier_round']
    n_outliers = metavals['n_outliers']
    #
    print(f'Round {outlier_round}: {n_outliers} outliers identified')
    print(f'Step 1: PLS regression on {vi_name} using all parameters')
    print()
    #
    # find n_components
    passnum = 1
    n_components, fit_error_metrics, betas, intercept, vip = \
        pls_regression(metavals, wxcd_detrended_stnd, vi_detrended_stnd,
                       passnum, n_components=0)
    #
    # use n_components to run PLS without VIP threshold
    initial_vip_df, n_components, fit_error_metrics, betas, intercept, vip = \
        vip_threshold_plsr(metavals, wxcd_detrended_stnd, vi_detrended_stnd,
                           wxcd_std_anom_vars, passnum, n_components=n_components,
                           vip_threshold=0.0, print_stats=True)
    # fit_error_metrics = [bias, mae, rmse, sse, rsq, aic, aicc, bic]
    #
    # compile initial results with full variable complement
    n_obs = metavals['n_observations'] - metavals['n_outliers']
    plsr_results = compile_step_results(plsr_results, 'initial', vi_name, n_obs,
                                        outlier_round, n_outliers, wxcd_std_anom_vars,
                                        n_components, vip, betas, intercept,
                                        fit_error_metrics)
    #
    return plsr_results


#
# end contents of phenology_model_plsr_step_1.py

**Notebook Cell 46**: invoke Step 1 in the iterative PLS modeling loop

Calls functions in **Notebook Cell 45**

In [ ]:
# part of model_vi_phenology.py
#

#
# gather input and response variables
wxcd_detrended_stnd = get_wxcd_detrended_stnd(vi_name, wxcd_std_anom_vars,
                                              pheno_fit_residuals_df)
vi_detrended_stnd = get_vi_detrended_stnd(vi_name, pheno_fit_residuals_df)
#
# PLS regression, Step 1
plsr_results_1 = plsr_step_1(metavals, plsr_results, wxcd_std_anom_vars,
                             wxcd_detrended_stnd, vi_detrended_stnd)


#
# end part of model_vi_phenology.py

---

**Notebook Cell 47**: Step 2 in the iterative PLS modeling loop, to find the most important WxCD variables

Calls functions in **Notebook Cells 43 and 44**

In [ ]:
# contents of phenology_model_plsr_step_2.py
#


def plsr_step_2(metavals, plsr_results, wxcd_detrended_stnd, vi_detrended_stnd):
    #
    # pull needed metavals
    vi_name = metavals['vi_name']
    outlier_round = metavals['outlier_round']
    vip_threshold = metavals['vip_threshold']
    n_outliers = metavals['n_outliers']
    #
    print(f'Round {outlier_round}: {n_outliers} outliers identified')
    print('Step 2: PLS regression to find the most important variables/parameters')
    print()
    #
    # use n_components found above to run PLS with VIP threshold
    wxcd_vars = plsr_results[f'{vi_name}_round_{outlier_round}_initial_vars']
    n_components = plsr_results[f'{vi_name}_round_{outlier_round}_initial_n_components']
    passnum = 1
    intermediate_vip_df, n_components, fit_error_metrics, betas, intercept, vip = \
        vip_threshold_plsr(metavals, wxcd_detrended_stnd, vi_detrended_stnd, wxcd_vars,
                           passnum, n_components=n_components, vip_threshold=vip_threshold,
                           print_stats=False)
    # fit_error_metrics = [bias, mae, rmse, sse, rsq, aic, aicc, bic]
    #
    # show top variable rankings (VIP and beta coefficients)
    print(f'{vi_name} PLS regression parameters with VIP >= {vip_threshold:.2f}')
    intermediate_vip_df.sort_values(by='vip', ascending=False, inplace=True)
    intermediate_vip_df.reset_index(drop=True, inplace=True)
    print(intermediate_vip_df)
    print()
    #
    # compile intermediate results for retained variables
    n_obs = metavals['n_observations'] - metavals['n_outliers']
    plsr_results = compile_step_results(plsr_results, 'intermediate', vi_name,
                                        n_obs, outlier_round, n_outliers, 
                                        list(intermediate_vip_df['varname']),
                                        n_components,
                                        list(intermediate_vip_df['vip']),
                                        list(intermediate_vip_df['beta']),
                                        intercept, fit_error_metrics)
    #
    return plsr_results


#
# end contents of phenology_model_plsr_step_2.py

**Notebook Cell 48**: invoke Step 2 in the iterative PLS modeling loop

Calls function in **Notebook Cell 47**

In [ ]:
# part of model_vi_phenology.py
#

#
# PLS regression, Step 2
plsr_results_2 = plsr_step_2(metavals, plsr_results_1,
                             wxcd_detrended_stnd, vi_detrended_stnd)


#
# end part of model_vi_phenology.py

---

**Notebook Cell 49**: Step 3 in the iterative PLS modeling loop, to find the PLS model using only the most important WxCD variables

Calls functions in **Notebook Cells 42, 43 and 44**

In [ ]:
# contents of phenology_model_plsr_step_3.py
#


def reduce_wxcd_vars(metavals, plsr_results, vi_detrended_stnd, pheno_fit_residuals_df):
    vi_name = metavals['vi_name']
    outlier_round = metavals['outlier_round']
    #
    retained_wxcd_vars = plsr_results[f'{vi_name}_round_{outlier_round}_intermediate_vars']
    retained_wxcd_detrended_stnd = \
        np.zeros((len(vi_detrended_stnd), len(retained_wxcd_vars)))
    #
    for n, var in enumerate(retained_wxcd_vars):
        if vi_name in var:
            var_name = f'{var}_detrended_stnd'
        else:
            var_name = f'{var}_detrended_stnd'
        var_arr = np.array(pheno_fit_residuals_df[var_name])
        if var_arr.ndim > 1:
            retained_wxcd_detrended_stnd[:, n] = var_arr[:, 0]
        else:
            retained_wxcd_detrended_stnd[:, n] = var_arr
    #
    return retained_wxcd_vars, retained_wxcd_detrended_stnd
    

def plsr_step_3(metavals, plsr_results, pheno_fit_residuals_df, vi_detrended_stnd):
    #
    # pull needed metavals
    vi_name = metavals['vi_name']
    outlier_round = metavals['outlier_round']
    vip_threshold = metavals['vip_threshold']
    n_outliers = metavals['n_outliers']
    #
    print(f'Round {outlier_round}: {n_outliers} outliers identified')
    print(f'Step 3: PLSR using only parameters with VIP >= {vip_threshold}')
    print()
    #
    # reduce the variable set to just the retained parameters
    retained_wxcd_vars, retained_wxcd_detrended_stnd = \
        reduce_wxcd_vars(metavals, plsr_results, vi_detrended_stnd, pheno_fit_residuals_df)
    #
    # find nc for the final set of retained variables
    passnum = 2
    n_components, fit_error_metrics, betas, intercept, vip = \
        pls_regression(metavals, retained_wxcd_detrended_stnd, vi_detrended_stnd, 
                       passnum, n_components=0)
    #
    # use nc found above to run PLS without VIP threshold
    final_vip_df, n_components, fit_error_metrics, betas, intercept, vip = \
        vip_threshold_plsr(metavals, retained_wxcd_detrended_stnd,
                           vi_detrended_stnd, retained_wxcd_vars, passnum,
                           n_components=n_components, vip_threshold=0.0,
                           print_stats=True)
    # fit_error_metrics = [bias, mae, rmse, sse, rsq, aic, aicc, bic]
    #
    # compile final results with reduced variable complement
    n_obs = metavals['n_observations'] - metavals['n_outliers']
    plsr_results = compile_step_results(plsr_results, 'final', vi_name, n_obs,
                                        outlier_round, n_outliers, 
                                        list(final_vip_df['varname']), 
                                        n_components, list(final_vip_df['vip']),
                                        list(final_vip_df['beta']), intercept,
                                        fit_error_metrics)
    #
    return plsr_results


#
# end contents of phenology_model_plsr_step_3.py

**Notebook Cell 50**: invoke Step 3 in the iterative PLS modeling loop

Calls function in **Notebook Cell 49**

In [ ]:
# part of model_vi_phenology.py
#

#
# PLS regression, Step 3
plsr_results_3 = plsr_step_3(metavals, plsr_results_2,
                             pheno_fit_residuals_df, vi_detrended_stnd)


#
# end part of model_vi_phenology.py

**Notebook Cell 51**: save the PLS modeling summary information for this round

In [ ]:
# contents of phenology_model_plsr_save_results.py
#


def save_plsr_results(metavals, plsr_results):
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    x_basis = metavals['x_basis']
    curve_str = metavals['vi_curve_str']
    outlier_round = metavals['outlier_round']
    fname_tuple = (point_id, vi_name, curve_str, x_basis, outlier_round)
    #
    outfname = '%s_pls_retained_%s_%s_%s_round_%d_results.csv' % fname_tuple
    outfpath = f'{output_path}/{outfname}'
    with open(outfpath, 'w') as outfile:
        w = csv.writer(outfile)
        for key, val in plsr_results.items():
            w.writerow([key, val])
    print(f'saved {outfname}')
    print()
    #
    return


#
# end contents of phenology_model_plsr_save_results.py

**Notebook Cell 52**: invoke procedure to save the PLS modeling summary for this round

Calls function in **Notebook Cell 51**

In [ ]:
# part of model_vi_phenology.py
#

#
# save results
save_plsr_results(metavals, plsr_results_3)


#
# end part of model_vi_phenology.py

---

### **PLS Prediction of Phenological Residuals**

*Description and details here!*

**Notebook Cell 53**: plot PLS-based predictions of VI values against observations

In [ ]:
# contents of phenology_model_plsp_plot.py
#


def pls_prediction_plot(vi_name, outlier_round, x_vals, y_vals, x_label, pred_type, title_label):
    regression_stats = regress(x_vals, y_vals)
    rsq = regression_stats[2]**2
    plt.scatter(x_vals, y_vals, marker='x', c='k', s=30)
    min_val = np.min([np.min(x_vals), np.min(y_vals)])
    max_val = np.max([np.max(x_vals), np.max(y_vals)])
    plt.plot([min_val, max_val], [min_val, max_val], 'k--', linewidth=1)
    x_regression = np.arange(np.round(min_val, 2), np.round(max_val, 2) + 0.01, 0.01)
    y_regression = regression_stats[0] * x_regression + regression_stats[1]
    plt.plot(x_regression, y_regression, 'b-', linewidth=2)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel(f'{x_label} {vi_name}', fontsize=11)
    plt.ylabel(f'Observed {vi_name}', fontsize=11)
    title_str = f'{pred_type} {vi_name} by {title_label} '
    title_str += f'(round {outlier_round}, n = {len(y_vals)}, Rsq = {rsq:.3f})'
    plt.title(title_str, fontsize=12)
    plt.tight_layout()
    #
    return


#
# end contents of phenology_model_plsp_plot.py

**Notebook Cell 54**: Step 4 in the iterative PLS modeling loop, to predict VI values using the parsimonious model

Calls functions in **Notebook Cells 8, 35, 36, 39, and 53**

In [ ]:
# contents of phenology_model_plsp.py
#


def plsp_image_dates(metavals, plsr_results, pheno_fit_params_df,
                     pheno_fit_residuals_df, trend_params_df, stnd_params_df):
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    x_basis = metavals['x_basis']
    curve_str = metavals['vi_curve_str']
    outlier_round = metavals['outlier_round']
    remove_outliers = metavals['remove_outliers']
    n_outliers = metavals['n_outliers']
    doymin = metavals['doy_min']
    doymax = metavals['doy_max']
    forecast_begin_year = metavals['forecast_begin_year']
    graph_output = metavals['graph_output']
    fname_tuple = (point_id, vi_name, curve_str, x_basis, outlier_round)
    #
    print(f'Round {outlier_round}: {n_outliers} outliers identified')
    print('Step 4: PLS prediction on image dates')
    print()
    #
    wxcd_reduced_vars = plsr_results[f'{vi_name}_round_{outlier_round}_final_vars']
    wxcd_reduced_nvars = len(wxcd_reduced_vars)
    n_components = plsr_results[f'{vi_name}_round_{outlier_round}_final_n_components']
    print(f'Predicting {vi_name} on all image dates using fitted PLS model')
    print(f'  ({wxcd_reduced_nvars} parameters, {n_components} components)')
    print()
    #
    retained_dates = np.array(pheno_fit_residuals_df['Date'])
    n_retained_dates = len(retained_dates)
    retained_doys = np.array(pheno_fit_residuals_df['DOY'])
    retained_dyears = np.array(pheno_fit_residuals_df['DYear'])
    #
    # prepare variables for PLS prediction
    vi_retained = np.array(pheno_fit_residuals_df[vi_name])
    vi_retained_detrended_stnd = np.array(pheno_fit_residuals_df[f'{vi_name}_residual_detrended_stnd'])
    wxcd_retained_detrended_stnd = np.zeros((n_retained_dates, wxcd_reduced_nvars))
    for n, var in enumerate(wxcd_reduced_vars):
        if vi_name in var:
            #
            # get vi_fitted for retained dates (detrended and standardized)
            var_arr = np.array(pheno_fit_residuals_df[f'{var}_detrended_stnd'])
            wxcd_retained_detrended_stnd[:, n] = var_arr
            #
            # then get and process vi_fitted for all image dates
            fitted_curve_params = np.array(pheno_fit_params_df[f'{vi_name}_by_{x_basis}'])
            retained_doys_ext = np.append(retained_doys, [doymin, doymax])
            vi_retained = get_abc_curve(retained_doys_ext, fitted_curve_params)
            #
            # detrend series using existing parameters
            detrend_params = np.array(trend_params_df[var][:2])
            vi_retained_detrended = detrend_given_stats(retained_dyears, vi_retained, detrend_params)
            #
            # standardize detrended series using existing parameters
            stnd_params = np.array(stnd_params_df[f'{var}_detrended'])
            vi_retained_detrended_stnd = standardize_given_stats(vi_retained_detrended, stnd_params)
            wxcd_retained_detrended_stnd[:, n] = vi_retained_detrended_stnd
        else:
            #
            # first gather detrended and standardized WxCD anoms for retained dates
            var_arr = np.array(pheno_fit_residuals_df[f'{var}_detrended_stnd'])
            if var_arr.ndim > 1:
                wxcd_retained_detrended_stnd[:, n] = var_arr[:, 0]
            else:
                wxcd_retained_detrended_stnd[:, n] = var_arr
            #
            # then get and process WxCD anoms for all image dates
            clim_anom = np.array(pheno_fit_residuals_df[var])
            #
            # detrend climatological residuals using existing parameters
            detrend_params = np.array(trend_params_df[var][:2])
            clim_anom_detrended = detrend_given_stats(retained_dyears, clim_anom, detrend_params)
            #
            # standardize detrended climatological residuals using existing parameters
            stnd_params = np.array(stnd_params_df[f'{var}_detrended'])
            clim_anom_detrended_stnd = standardize_given_stats(clim_anom_detrended, stnd_params)
            wxcd_retained_detrended_stnd[:, n] = clim_anom_detrended_stnd
    #
    # PLS prediction based on regression model (derived above, repeating for accuracy)
    vi_retained_detrended_stnd_predicted, _, coeffs, intercept, vip = \
        pls_prediction(wxcd_retained_detrended_stnd, vi_retained_detrended_stnd,
                       wxcd_retained_detrended_stnd, n_components)
    #
    # destandardize + retrend retained predicted VI residuals
    stnd_params = np.array(stnd_params_df[f'{vi_name}_residual_detrended'])
    vi_retained_detrended_predicted = \
        destandardize(vi_retained_detrended_stnd_predicted, stnd_params)
    detrend_params = np.array(trend_params_df[f'{vi_name}_residual'][:2])
    vi_retained_predicted_residual = \
        retrend(retained_dyears, vi_retained_detrended_predicted, detrend_params)
    #
    # recombine retained predicted VI residuals with mean phenology curve
    fitted_curve_params = np.array(pheno_fit_params_df[f'{vi_name}_by_{x_basis}'])
    doys_ext = np.append(retained_doys, [doymin, doymax])
    vi_retained_mean = get_abc_curve(doys_ext, fitted_curve_params)
    vi_retained_predicted = vi_retained_mean + vi_retained_predicted_residual
    col_name = f'{vi_name}_by_{x_basis}_predicted_round_{outlier_round}'
    pheno_fit_residuals_df_new_cols = pd.DataFrame({col_name: vi_retained_predicted})
    pheno_fit_residuals_df = pd.concat([pheno_fit_residuals_df, pheno_fit_residuals_df_new_cols], axis=1)
    #
    if graph_output:
        if remove_outliers or forecast_begin_year:
            plt.figure(figsize=(12, 12))
        else:
            plt.figure(figsize=(12, 6))
        #
        # plot figure with retained observations vs mean-curve predictions
        if remove_outliers or forecast_begin_year:
            plt.subplot(2, 2, 1)
        else:
            plt.subplot(1, 2, 1)
        pls_prediction_plot(vi_name, outlier_round, vi_retained_mean, vi_retained,
                            'Phenological mean', 'Expected', 'mean curve')
        #
        # plot figure with retained observations vs predictions
        if remove_outliers or forecast_begin_year:
            plt.subplot(2, 2, 2)
        else:
            plt.subplot(1, 2, 2)
        pls_prediction_plot(vi_name, outlier_round, vi_retained_predicted, vi_retained,
                            'PLS-predicted', 'Expected', 'PLS')
    #
    # destandardize + retrend predicted VI residuals
    stnd_params = np.array(stnd_params_df[f'{vi_name}_residual_detrended'])
    vi_retained_detrended_predicted = destandardize(vi_retained_detrended_stnd_predicted, stnd_params)
    detrend_params = np.array(trend_params_df[f'{vi_name}_residual'][:2])
    vi_retained_predicted_residual = retrend(retained_dyears, vi_retained_detrended_predicted, detrend_params)
    #
    # recombine predicted VI residuals with base curve
    fitted_curve_params = np.array(pheno_fit_params_df[f'{vi_name}_by_{x_basis}'])
    retained_doys_ext = np.append(retained_doys, [doymin, doymax])
    vi_retained_mean = get_abc_curve(retained_doys_ext, fitted_curve_params)
    vi_retained_predicted = vi_retained_mean + vi_retained_predicted_residual
    #
    # expand VI dataframe with PLS-predicted values
    pheno_fit_residuals_df[f'{vi_name}_mean'] = np.array(vi_retained_mean)
    pheno_fit_residuals_df[f'{vi_name}_predicted'] = np.array(vi_retained_predicted)
    #
    if graph_output:
        if forecast_begin_year:
            #
            # trim dataframe to forecast period
            forecast_wxcd_vi_df = pheno_fit_residuals_df[pheno_fit_residuals_df['Year'] >= forecast_begin_year]        
            vi_forecast = np.array(forecast_wxcd_vi_df[vi_name])
            vi_forecast_mean = np.array(forecast_wxcd_vi_df[f'{vi_name}_mean'])
            vi_forecast_predicted = np.array(forecast_wxcd_vi_df[f'{vi_name}_predicted'])
            #
            # plot forecast period figure with observations vs mean-curve predictions
            plt.subplot(2, 2, 3)
            pls_prediction_plot(vi_name, outlier_round, vi_forecast_mean, vi_forecast,
                                'Phenological mean', 'Forecast', 'mean curve')
            #
            # plot forecast period figure with observations vs PLS predictions
            plt.subplot(2, 2, 4)
            pls_prediction_plot(vi_name, outlier_round, vi_forecast_predicted, vi_forecast,
                                'PLS-predicted', 'Expected', 'PLS')
            #
            plotfname = '%s_%s_%s_%s_round_%d_pls_prediction+forecast.png' % fname_tuple
            plotfpath = f'{output_path}/{plotfname}'
            plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
            print(f'saved {plotfname}')
        else:
            plotfname = '%s_%s_%s_%s_round_%d_pls_prediction.png' % fname_tuple
            plotfpath = f'{output_path}/{plotfname}'
            plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
            print(f'saved {plotfname}')
    #
    # save expanded VI dataframe with PLS predicted values
    #
    outfname = '%s_pls_predicted_%s_%s_%s_round_%d.csv' % fname_tuple
    outfpath = f'{output_path}/{outfname}'
    pheno_fit_residuals_df.to_csv(outfpath, index=False)
    print(f'saved {outfname}')
    #
    return pheno_fit_residuals_df


#
# end contents of phenology_model_plsp.py

**Notebook Cell 55**: invoke procedure to predict VI values using the parsimonious model

Calls function in **Notebook Cell 54**

In [ ]:
# part of model_vi_phenology.py
#

#
# do PLS model-based prediction on image dates
pheno_fit_residuals_df = plsp_image_dates(metavals, plsr_results_3, pheno_fit_params_df,
                                          pheno_fit_residuals_df, trend_params_df, stnd_params_df)


#
# end part of model_vi_phenology.py

---

**Notebook Cell 56**: calculate and plot the statistical prediction interval for the parsimonious PLS-based model, and identify outlier observations

In [ ]:
# contents of phenology_model_get_outliers.py
#


def calculate_prediction_interval(x, y, outlier_confidence):
    #
    # this function is based on
    # https://github.com/demotu/BMC/blob/master/notebooks/CurveFitting.ipynb
    #
    n_obs = y.size
    parameters, covariance = np.polyfit(x, y, 1, cov=True)
    y_fit = np.polyval(parameters, x)
    residuals = y - y_fit                           
    n_params = parameters.size
    degrees_of_freedom = n_obs - n_params
    stdv_of_error = np.sqrt(np.sum(residuals**2) / degrees_of_freedom)
    p_value = 1.0 - ((1.0 - outlier_confidence) / 2.0)
    t_value = scipy.stats.t.ppf(p_value, degrees_of_freedom)
    num = (x - np.mean(x))**2
    denom = np.sum((x - np.mean(x))**2)
    radicand = 1 + (1 / n_obs) + (num / denom)  # for prediction interval, not confidence interval
    prediction_interval = t_value * stdv_of_error * np.sqrt(radicand)
    #
    return y_fit, prediction_interval


def pls_prediction_interval_plot(metavals, vi_retained, vi_retained_predicted, retained_dates):
    #
    # this function is based on
    # https://stackoverflow.com/questions/27164114/show-confidence-limits-and-prediction-limits-in-scatter-plot
    # which is itself based on 
    # https://github.com/demotu/BMC/blob/master/notebooks/CurveFitting.ipynb    
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    x_basis = metavals['x_basis']
    curve_str = metavals['vi_curve_str']
    outlier_round = metavals['outlier_round']
    outlier_confidence = metavals['outlier_confidence']
    graph_output = metavals['graph_output']
    fname_tuple = (point_id, vi_name, curve_str, x_basis, outlier_round)
    #
    # sort the values on x to plot the prediction interval correctly
    df = pd.DataFrame({'x': vi_retained_predicted,
                       'y': vi_retained,
                       'd': retained_dates})
    df.sort_values(by=['x'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    x = np.array(df['x'])
    y = np.array(df['y'])
    dates = list(df['d'])
    #
    if graph_output:
        plt.figure(figsize=(6, 6))
        #
        # plot measured and predicted VI for individual retained dates
        plt.scatter(x, y, marker='x', c='k', s=30,
                    label=f'Landsat scenes ({len(vi_retained)})')
        #
        # plot 1:1 line
        min_val = np.min([np.min(x), np.min(y)])
        max_val = np.max([np.max(x), np.max(y)])
        plt.plot([min_val-0.02, max_val+0.02], [min_val-0.02, max_val+0.02], 
                 'k--', linewidth=1, label='1:1 line')
        #
        # plot regression line
        regression_stats = regress(x, y)
        rsq = regression_stats[2]**2
        x_regression = np.arange(np.round(min_val, 2), np.round(max_val, 2) + 0.01, 0.01)
        y_regression = regression_stats[0] * x_regression + regression_stats[1]
        rsq_str = r'$r^2$'
        plt.plot(x_regression, y_regression, 'b-', linewidth=2,
                 label=f'linear regression ({rsq_str} = {rsq:.4f})')
    #
    # calculate and plot the prediction interval based on the stated outlier confidence
    y_fit, prediction_interval = \
        calculate_prediction_interval(x, y, outlier_confidence)
    if graph_output:
        confidence_str = f'{(outlier_confidence * 100):.1f}%'
        plt.fill_between(x, y_fit+prediction_interval, y_fit-prediction_interval,
                         color='lightblue', alpha=0.5,
                         label=f'{confidence_str} prediction interval')
        #
        # overplot measured and predicted VI for individual retained dates
        plt.scatter(vi_retained_predicted, vi_retained, marker='x', c='k', s=30)
    #
    # identify outliers
    outlier_x = []
    outlier_y = []
    outlier_dates = []
    y_dist = vi_retained - y_fit
    for i, x_i in enumerate(x):
        y_i = y[i]
        y_fit_i = y_fit[i]
        dist = np.abs(y_i - y_fit_i)
        pi_i = prediction_interval[i]
        if dist > pi_i:
            outlier_x.append(x_i)
            outlier_y.append(y_i)
            outlier_dates.append(dates[i])
    if graph_output:
        plt.scatter(outlier_x, outlier_y, marker='o', c='r', edgecolor='k', s=40,
                    label=f'outlier scenes ({len(outlier_y)})')
        #
        plt.legend(loc='upper left')
        #
        plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)
        plt.xlabel(f'PLS-predicted {vi_name}', fontsize=11)
        plt.ylabel(f'Observed {vi_name}', fontsize=11)
        title = f'Round {outlier_round} predicted {vi_name} by PLS'
        plt.title(title, fontsize=12)
        plt.tight_layout()
        #
        plotfname = '%s_%s_%s_%s_round_%d_pls_prediction_outliers.png' % fname_tuple
        plotfpath = f'{output_path}/{plotfname}'
        plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
        print(f'saved {plotfname}')
    #
    return outlier_dates
    

def find_outliers(metavals, pheno_fit_residuals_df, plot_outliers=True):
    #
    # pull needed metavals
    vi_name = metavals['vi_name']
    x_basis = metavals['x_basis']
    outlier_round = metavals['outlier_round']
    #
    # plot figure with retained observations vs predictions
    vi_retained = np.array(pheno_fit_residuals_df[vi_name])
    col_name = f'{vi_name}_by_{x_basis}_predicted_round_{outlier_round}'
    vi_retained_predicted = np.array(pheno_fit_residuals_df[col_name])
    retained_dates = np.array(pheno_fit_residuals_df['Date'])
    #
    if plot_outliers:
        outlier_dates = pls_prediction_interval_plot(metavals, vi_retained,
                                                     vi_retained_predicted,
                                                     retained_dates) 
    #
    return outlier_dates


#
# end contents of phenology_model_get_outliers.py

**Notebook Cell 57**: invoke procedure to identify outlier observations using the statistical prediction interval for the parsimonious PLS-based model

Calls function in **Notebook Cell 56**

In [ ]:
# part of model_vi_phenology.py
#


if metavals['remove_outliers']:
    outlier_dates = find_outliers(metavals, pheno_fit_residuals_df)
    print('End of Round %d: post-PLS outlier identification' % metavals['outlier_round'])
    if outlier_dates:
        confidence = metavals['outlier_confidence']
        confidence_str = f'{(confidence * 100):.1f}%'
        print(f'- found {len(outlier_dates)} outlier dates based on {confidence_str} prediction interval')
        print(f'{str(outlier_dates)}')
    else:
        print('- no outliers found')
else:
    print('No outlier iterations requested')
    metavals['optimal_outlier_round'] = 0
print()


#
# end part of model_vi_phenology.py

---

**Notebook Cell 58**: plot a summary of PLS model statistics through several rounds of outlier removal and optimization

In [ ]:
# contents of phenology_model_plsr_summary.py
#


def make_summary_plot(x_vals, y_vals, y_label, vi_name, title_end):
    plt.plot(x_vals, y_vals, 'b-', marker='o', linewidth=2)
    plt.xlabel('Outlier Round', fontsize=11)
    plt.ylabel(y_label, fontsize=11)
    plt.xticks(x_vals, fontsize=10)
    plt.yticks(fontsize=10)
    plt.title(f'{vi_name} PLSR model summary {title_end}')
    plt.tight_layout()
    #
    return


def plot_plsr_summary(metavals, plsr_results):
    #
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    x_basis = metavals['x_basis']
    curve_str = metavals['vi_curve_str']
    n_outlier_rounds = metavals['outlier_round']
    fname_tuple = (point_id, vi_name, curve_str, x_basis)
    outlier_rounds = np.arange(n_outlier_rounds+1)
    #
    plt.figure(figsize=(12, 9))
    #
    plt.subplot(3, 3, 1)
    round_final_n_obs = [plsr_results[f'{vi_name}_round_{r}_final_n_obs'] for r in outlier_rounds]
    make_summary_plot(outlier_rounds, round_final_n_obs, 'n_observations remaining', vi_name, 'n_observations')
    #
    plt.subplot(3, 3, 2)
    round_final_n_outliers = [plsr_results[f'{vi_name}_round_{r}_final_n_outliers'] for r in outlier_rounds]
    make_summary_plot(outlier_rounds, round_final_n_outliers, 'total n_outliers identified', vi_name, 'n_outliers')
    #
    err_metrics = ['AICc', 'PRESS', 'Rsq', 'RMSE', 'MAE', 'Nvars', 'n_components']
    for m, metric in enumerate(err_metrics):
        plt.subplot(3, 3, m+3)
        round_final_metric = [plsr_results[f'{vi_name}_round_{r}_final_{metric.lower()}'] for r in outlier_rounds]
        make_summary_plot(outlier_rounds, round_final_metric, f'Round final {metric}', vi_name, metric)
    #
    plotfname = '%s_%s_%s_%s_pls_modeling_stats_summary.png' % fname_tuple
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    #
    return


#
# end contents of phenology_model_plsr_summary.py

**Notebook Cell 59**: main modeling function to execute several rounds of outlier removal and PLS-based model optimization

Calls functions in **Notebook Cells 14, 17, 37, 45, 47, 49, 51, 54, 56, and 58**

In [ ]:
# contents of phenology_model_remove_outliers.py
#


def remove_outliers(metavals, outlier_dates, plsr_results, wxcd_vi_retained_df):
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    x_basis = metavals['x_basis']
    curve_str = metavals['vi_curve_str']
    remove_outliers = metavals['remove_outliers']
    max_outlier_rounds = metavals['max_outlier_rounds']
    graph_output = metavals['graph_output']
    #
    outlier_round = metavals['outlier_round']
    if outlier_round >= metavals['max_outlier_rounds']:
        iterate = False
    else:
        iterate = True
    overall_aicc = [plsr_results[f'{vi_name}_round_{outlier_round}_final_aicc']]
    disturb_dates_all = []
    #
    while iterate:
        #
        # pull needed metavals
        outlier_round = metavals['outlier_round']
        #
        fname_tuple = (point_id, vi_name, curve_str, x_basis, outlier_round)
        #
        # concatenate outlier collections
        disturb_dates_all = sorted(list(set(disturb_dates_all) | set(outlier_dates)))
        #
        # update metavals
        metavals['n_outliers'] = len(disturb_dates_all)
        #
        # split dataset according to found outliers
        wxcd_vi_outliers_df = wxcd_vi_retained_df[wxcd_vi_retained_df.Date.isin(outlier_dates)]
        nrows, ncols = wxcd_vi_outliers_df.shape
        print(f'outliers WxCD/VI data array has {nrows} rows, {ncols} cols')
        #
        # save outlier dates information
        outfname = '%s_outliers_%s_%s_%s_round_%d.csv' % fname_tuple
        outfpath = f'{output_path}/{outfname}'
        wxcd_vi_outliers_df.to_csv(outfpath, index=False)
        print(f'- saved {outfname}')
        print()
        #
        # reduce dataset to retained dates
        wxcd_vi_retained_df = wxcd_vi_retained_df[~wxcd_vi_retained_df.Date.isin(outlier_dates)]
        nrows, ncols = wxcd_vi_retained_df.shape
        print(f'retained WxCD/VI data array has {nrows} rows, {ncols} cols')
        #
        # save retained dates information
        outfname = '%s_retained_%s_%s_%s_round_%d.csv' % fname_tuple
        outfpath = f'{output_path}/{outfname}'
        wxcd_vi_retained_df.to_csv(outfpath, index=False)
        print(f'- saved {outfname}')
        print()
        #
        # fit ABC curve
        metavals['outlier_round'] += 1
        pheno_fit_params_df, pheno_fit_residuals_df, full_year_mean_vi = \
            model_vi_pheno_curve(metavals, wxcd_vi_retained_df, wxcd_vi_outliers_df)
        #
        # get ABC curve metrics
        pheno_fit_metrics_df = get_vi_fit_metrics(metavals, pheno_fit_params_df)
        #
        # PLSR prep
        trend_params_df, stnd_params_df, pheno_fit_residuals_df = \
            plsr_prep(metavals, wxcd_std_anom_vars, wxcd_vi_retained_df, pheno_fit_residuals_df)
        #
        # gather input and response variables
        wxcd_detrended_stnd = get_wxcd_detrended_stnd(vi_name, wxcd_std_anom_vars,
                                                      pheno_fit_residuals_df)
        vi_detrended_stnd = get_vi_detrended_stnd(vi_name, pheno_fit_residuals_df)        
        #
        # PLSR Step 1
        plsr_results_1 = plsr_step_1(metavals, plsr_results, wxcd_std_anom_vars,
                                     wxcd_detrended_stnd, vi_detrended_stnd)
        #
        # PLSR Step 2
        plsr_results_2 = plsr_step_2(metavals, plsr_results_1, wxcd_detrended_stnd,
                                     vi_detrended_stnd)
        #
        # PLSR Step 3
        plsr_results_3 = plsr_step_3(metavals, plsr_results_2, pheno_fit_residuals_df,
                                     vi_detrended_stnd)
        #
        # save results
        save_plsr_results(metavals, plsr_results_3)
        #
        # Check model AICc
        last_round = metavals['outlier_round'] - 1
        last_aicc = overall_aicc[-1]
        print(f'Round {last_round} final AICc: {last_aicc:.1f}')
        this_round = metavals['outlier_round']
        this_aicc = plsr_results_3[f'{vi_name}_round_{str(metavals["outlier_round"])}_final_aicc']
        print(f'Round {this_round} final AICc: {this_aicc:.1f}')
        #
        # compare press values and exit iterations if beyond the minimum
        if this_aicc < overall_aicc[-1]:
            iterate = True
            print('- continuing outlier iterations')
            overall_aicc.append(this_aicc)
        else:
            iterate = False
            print('- ending outlier iterations')
            print()
            break
        print()
        #
        # PLS Prediction on image dates
        pheno_fit_residuals_df = plsp_image_dates(metavals, plsr_results_3, pheno_fit_params_df,
                                                  pheno_fit_residuals_df, trend_params_df, stnd_params_df)
        #
        # If max_outlier_rounds reached
        if outlier_round >= max_outlier_rounds:
            iterate = False
            print('max_outlier_rounds reached - ending outlier iterations')
            print()
            break
        #
        # Check for outliers
        outlier_dates = find_outliers(metavals, pheno_fit_residuals_df)
        print()
        print(f'End of Round {this_round}: post-PLS outlier identification')
        if outlier_dates:
            confidence = metavals['outlier_confidence']
            confidence_str = f'{(confidence * 100):.1f}%'
            print(f'- found {len(outlier_dates)} outlier dates based on {confidence_str} prediction interval')
            print(f'{str(outlier_dates)}')
            print()
        else:
            print('no more outliers found - ending outlier iterations')
            iterate = False
            outfname = f'{point_id}_outliers_{vi_name}_{curve_str}_{x_basis}_round_{this_round}.csv'
            outfpath = f'{output_path}/{outfname}'
            wxcd_vi_outliers_df.to_csv(outfpath, index=False)
            print(f'- saved {outfname}')            
            print()
            break
    if graph_output:
        plot_plsr_summary(metavals, plsr_results_3)
    #
    min_aicc = np.min(overall_aicc)
    optimal_round = np.argmin(overall_aicc)
    metavals['optimal_outlier_round'] = optimal_round
    print(f'Outlier iterations optimized at {optimal_round} rounds')
    print(f'- minimum AICc = {min_aicc:.1f}')
    #
    outfname = f'{point_id}_metavals_{vi_name}_{curve_str}_{x_basis}.csv'
    outfpath = f'{output_path}/{outfname}'
    with open(outfpath, 'w') as outfile:
        w = csv.writer(outfile)
        for key, val in metavals.items():
            w.writerow([key, val])
    print(f'- saved {outfname}')
    print()
    #
    return


#
# end contents of phenology_model_remove_outliers.py

**Notebook Cell 60**: invoke the main model looping function, if outlier removal is indicated

Calls function in **Notebook Cell 59**

In [ ]:
# part of model_vi_phenology.py
#


if remove_outliers and outlier_dates:
    remove_outliers(metavals, outlier_dates, plsr_results, wxcd_vi_df)


#
# end part of model_vi_phenology.py

---

### Final model plotting + reporting

**Notebook Cell 61**: report the final mean phenology curve, its metrics, and the VI outlier observations

In [ ]:
#
#


def plot_vi_values_fitted_curve_outliers(vi_name, vi_doys, vi_values, doys_range,
                                         season_start, season_end, begin_year, end_year,
                                         full_year_mean_vi, var_fit_params, outlier_round,
                                         rsq, output_path, fname_tuple, vi_outliers_df):
    plt.figure(figsize=(6, 4))
    plt.scatter(vi_doys, vi_values, marker='x', c='k', s=30)
    outliers_doys = np.array(vi_outliers_df['DOY'])
    outliers_values = np.array(vi_outliers_df[vi_name])
    plt.scatter(outliers_doys, outliers_values, marker='o', c='r',
                edgecolor='k', s=30)
    plt.plot(doys_range, full_year_mean_vi, 'b-', linewidth=2)
    control_doys = [var_fit_params[2], var_fit_params[6],
                    var_fit_params[10], var_fit_params[12]]
    control_vi_values = [full_year_mean_vi[int(round(var_fit_params[2])) - 1],
                         var_fit_params[4], var_fit_params[8], 
                         full_year_mean_vi[int(round(var_fit_params[12])) - 1]]
    plt.scatter(control_doys, control_vi_values, marker='o', c='g', edgecolor='g', s=50)
    plt.xlim([season_start, season_end])
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel('DOY', fontsize=11)
    plt.ylabel(vi_name, fontsize=11)
    title = f'{begin_year}-{end_year} {vi_name} mean phenocurve '
    title += f'(n = {len(vi_values)}, Rsq = {rsq:.3f}, o = {len(outliers_values)})'
    plt.title(title, fontsize=12)
    plt.tight_layout()
    plotfname = '%s_%s_%s_%s_final_mean_fit+outliers.png' % fname_tuple
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    #
    return


print('Post-modeling summary')
print('Part 1: Mean phenology')
print()
#
# pull needed metavals
output_path = metavals['output_path']
point_id = metavals['point_id']
vi_name = metavals['vi_name']
curve_str = metavals['vi_curve_str']
x_basis = metavals['x_basis']
curve_fit_vars = abc_curve_fit_vars
begin_year = metavals['analysis_begin_year']
end_year = metavals['analysis_end_year']
outlier_round = metavals['outlier_round']
remove_outliers = metavals['remove_outliers']
n_outliers = metavals['n_outliers']
doy_min = metavals['doy_min']
doy_max = metavals['doy_max']
growing_season_start = metavals['growing_season_start']
growing_season_end = metavals['growing_season_end']
optimal_outlier_round = metavals['optimal_outlier_round']
graph_output = metavals['graph_output']
fname_tuple = (point_id, vi_name, curve_str, x_basis)
#
# get original input dataset
full_dataset_fname = f'{point_id}_input_WxCD_VI_cleaned_trimmed.csv'
full_dataset_fpath = f'{output_path}/{full_dataset_fname}'
print(f'reading {full_dataset_fname}')
wxcd_vi_all_df = pd.read_csv(full_dataset_fpath, index_col=None)
#
# get outliers
for outlier_round in range(optimal_outlier_round):
    outliers_fname = f'{point_id}_outliers_{vi_name}_{curve_str}_{x_basis}_round_{outlier_round}.csv'
    outliers_fpath = f'{output_path}/{outliers_fname}'
    print(f'reading {outliers_fname}')
    if not outlier_round:
        vi_outliers_df = pd.read_csv(outliers_fpath, index_col=None)
    else:
        vi_outliers_df = pd.concat([vi_outliers_df, pd.read_csv(outliers_fpath, index_col=None)], axis=0)
outliers_fname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_all_outliers.csv'
outliers_fpath = f'{output_path}/{outliers_fname}'
vi_outliers_df.to_csv(outliers_fpath, index=False)
print(f'saved {outliers_fname}')
#
# split dataset to retained and outlier dates
outlier_dates = list(vi_outliers_df['Date'])
wxcd_vi_retained_df = wxcd_vi_all_df[~wxcd_vi_all_df.Date.isin(outlier_dates)]
wxcd_vi_outliers_df = wxcd_vi_all_df[wxcd_vi_all_df.Date.isin(outlier_dates)]
vi_doys = np.array(wxcd_vi_retained_df[x_basis]).astype(int)
vi_values = np.array(wxcd_vi_retained_df[vi_name])
#
# get full-year mean phenology curve
infname = f'{point_id}_retained_{vi_name}_{curve_str}_{x_basis}_round_{optimal_outlier_round}_full_year.npy'
infpath = f'{output_path}/{infname}'
outfname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_mean_phenology_full_year.npy'
outfpath = f'{output_path}/{outfname}'
os.system(f'cp {infpath} {outfpath}')
print(f'reading {outfname}')
full_year_mean_vi = np.load(outfpath)
#
# get mean phenology curve fit parameters
infname = f'{point_id}_retained_{vi_name}_{curve_str}_{x_basis}_round_{optimal_outlier_round}_fit_params.csv'
infpath = f'{output_path}/{infname}'
outfname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_mean_phenology_fit_params.csv'
outfpath = f'{output_path}/{outfname}'
os.system(f'cp {infpath} {outfpath}')
print(f'reading {outfname}')
pheno_fit_params_df = pd.read_csv(outfpath, index_col=None)
vi_fit_params = pheno_fit_params_df[f'{vi_name}_by_{x_basis}']
rsq = vi_fit_params[curve_fit_vars.index('Rsq')]
#
doys_range = np.arange(doy_min, doy_max + 1)
if graph_output:
    plot_vi_values_fitted_curve_outliers(vi_name, vi_doys, vi_values, doys_range,
                                         growing_season_start, growing_season_end,
                                         begin_year, end_year, full_year_mean_vi,
                                         vi_fit_params, outlier_round, rsq,
                                         output_path, fname_tuple, wxcd_vi_outliers_df)
#
# get mean phenology curve fit metrics
infname = f'{point_id}_retained_{vi_name}_{curve_str}_{x_basis}_round_{optimal_outlier_round}_fit_metrics.csv'
infpath = f'{output_path}/{infname}'
outfname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_mean_phenology_fit_metrics.csv'
outfpath = f'{output_path}/{outfname}'
os.system(f'cp {infpath} {outfpath}')
print(f'reading {outfname}')
pheno_fit_metrics_df = pd.read_csv(outfpath, index_col=None)
print()
#
print('Mean phenology curve metrics')
print(pheno_fit_metrics_df)
print()


#
#

**Notebook Cell 62**: report the final PLS model variables and their VIP and beta values

In [ ]:
#
# 


print('Post-modeling summary')
print('Part 2: PLS regression results')
print()
#
# pull needed metavals
output_path = metavals['output_path']
point_id = metavals['point_id']
vi_name = metavals['vi_name']
curve_str = metavals['vi_curve_str']
x_basis = metavals['x_basis']
optimal_outlier_round = metavals['optimal_outlier_round']
#
# find final PLSR results
infname = f'{point_id}_pls_retained_{vi_name}_{curve_str}_{x_basis}_round_{optimal_outlier_round}_results.csv'
infpath = f'{output_path}/{infname}'
print(f'reading {infname}')
summary_data_df = pd.read_csv(infpath, index_col=None)
summary_data_df.columns = ['key', 'value']
var_data_df = \
    summary_data_df[summary_data_df['key'].str.contains(f'{vi_name}_round_{optimal_outlier_round}_final_')]
outfname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_PLSR_results.csv'
outfpath = f'{output_path}/{outfname}'
var_data_df.to_csv(outfpath, index=False)
print(f'wrote {outfname}')
#
# extract relevant PLSR results
regression_vars_str = var_data_df.loc[var_data_df.key==f'{vi_name}_round_{optimal_outlier_round}_final_vars',
                                      'value'].values[0]
regression_vars = ast.literal_eval(regression_vars_str)
regression_vip_str = var_data_df.loc[var_data_df.key==f'{vi_name}_round_{optimal_outlier_round}_final_vip',
                                     'value'].values[0]
regression_vip = ast.literal_eval(regression_vip_str)
regression_betas_str = var_data_df.loc[var_data_df.key==f'{vi_name}_round_{optimal_outlier_round}_final_betas',
                                       'value'].values[0]
regression_betas = ast.literal_eval(regression_betas_str)
#
# store final variables/VIP/betas
regression_results_df = pd.DataFrame({'var': regression_vars,
                                      'vip': regression_vip,
                                      'beta': regression_betas})
regression_results_df.sort_values(by=['vip'], ascending=False, inplace=True)
regression_results_df.reset_index(drop=True, inplace=True)
outfname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_PLSR_vars+betas.csv'
outfpath = f'{output_path}/{outfname}'
regression_results_df.to_csv(outfpath, index=False)
print(f'wrote {outfname}')
print()
#
print('PLS regression variables and beta coefficients')
print(regression_results_df)
print()


#
#

**Notebook Cell 63**: report the final PLS-based predicted VI values and observation anomalies, and plot time series

In [ ]:
#
#


def pls_prediction_regression_plot(metavals, prediction_all_df):
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    curve_str = metavals['vi_curve_str']
    x_basis = metavals['x_basis']
    #
    all_observed = prediction_all_df[vi_name]
    all_predicted = prediction_all_df[f'{vi_name}_predicted']
    retained_dates_df = prediction_all_df[prediction_all_df[f'{vi_name}_outlier'] == 0]
    outlier_dates_df = prediction_all_df[prediction_all_df[f'{vi_name}_outlier'] == 1]
    retained_observed = retained_dates_df[vi_name]
    retained_predicted = retained_dates_df[f'{vi_name}_predicted']
    outlier_observed = outlier_dates_df[vi_name]
    outlier_predicted = outlier_dates_df[f'{vi_name}_predicted']
    #
    plt.figure(figsize=(6, 6))
    #
    # plot retained dates
    plt.scatter(retained_predicted, retained_observed, marker='x', c='k', s=30,
                label=f'modeled dates ({len(retained_predicted)})')
    #
    # plot outlier dates
    plt.scatter(outlier_predicted, outlier_observed, marker='o', c='r', edgecolor='k', s=40,
                label=f'outlier dates ({len(outlier_predicted)})')
    #
    # plot 1:1 line
    min_val = np.min([np.min(all_predicted), np.min(all_observed)])
    max_val = np.max([np.max(all_predicted), np.max(all_observed)])
    plt.plot([min_val-0.02, max_val+0.02], [min_val-0.02, max_val+0.02], 
             'k--', linewidth=1, label='1:1 line')
    #
    # plot regression line
    regression_stats = regress(retained_predicted, retained_observed)
    rsq = regression_stats[2]**2
    x_regression = np.arange(np.round(min_val, 2), np.round(max_val, 2) + 0.01, 0.01)
    y_regression = regression_stats[0] * x_regression + regression_stats[1]
    rsq_str = r'$r^2$'
    plt.plot(x_regression, y_regression, 'b-', linewidth=2,
             label=f'linear regression')
    #
    # overplot retained dates
    plt.scatter(retained_predicted, retained_observed, marker='x', c='k', s=30)
    #
    plt.legend(loc='upper left')
    #
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel(f'PLS-predicted {vi_name}', fontsize=11)
    plt.ylabel(f'Observed {vi_name}', fontsize=11)
    title = f'Final {vi_name} predictions by PLS ({rsq_str} = {rsq:.3f})'
    plt.title(title, fontsize=12)
    plt.tight_layout()
    #
    plotfname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_PLS_prediction.png'
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    #
    return


def pls_prediction_time_series_plot(metavals, prediction_all_df):
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    curve_str = metavals['vi_curve_str']
    x_basis = metavals['x_basis']
    #
    all_dyear = prediction_all_df['DYear']
    retained_dates_df = prediction_all_df[prediction_all_df[f'{vi_name}_outlier'] == 0]
    outlier_dates_df = prediction_all_df[prediction_all_df[f'{vi_name}_outlier'] == 1]
    retained_dyear = retained_dates_df['DYear']
    retained_observed = retained_dates_df[vi_name]
    outlier_dyear = outlier_dates_df['DYear']
    outlier_observed = outlier_dates_df[vi_name]
    outlier_predicted = outlier_dates_df[f'{vi_name}_predicted']
    #
    plt.figure(figsize=(12, 4))
    #
    # plot retained dates
    plt.scatter(retained_dyear, retained_observed, marker='x', c='k', s=30,
                label=f'modeled observations ({len(retained_dyear)})')
    #
    # plot outlier observations
    plt.scatter(outlier_dyear, outlier_observed, marker='o', c='r', edgecolor='k', s=40,
                label=f'outlier observations ({len(outlier_dyear)})')
    #
    # plot outlier predictions
    plt.scatter(outlier_dyear, outlier_predicted, marker='x', c='b', s=30,
                label=f'outlier predictions ({len(outlier_dyear)})')
    #
    # plot zero line
    min_val = np.floor(np.min(all_dyear))
    max_val = np.ceil(np.max(all_dyear))
    plt.plot([min_val, max_val], [0.0, 0.0], 'k--', linewidth=1)
    #
    plt.legend(loc='upper left')
    #
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel(f'Date', fontsize=11)
    plt.ylabel(vi_name, fontsize=11)
    title = f'Landsat {vi_name} observation and prediction time series'
    plt.title(title, fontsize=12)
    plt.tight_layout()
    #
    plotfname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_time_series.png'
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    return


def pls_prediction_time_series_anomaly_plot(metavals, prediction_all_df):
    #
    # pull needed metavals
    output_path = metavals['output_path']
    point_id = metavals['point_id']
    vi_name = metavals['vi_name']
    curve_str = metavals['vi_curve_str']
    x_basis = metavals['x_basis']
    #
    all_dyear = prediction_all_df['DYear']
    retained_dates_df = prediction_all_df[prediction_all_df[f'{vi_name}_outlier'] == 0]
    outlier_dates_df = prediction_all_df[prediction_all_df[f'{vi_name}_outlier'] == 1]
    retained_dyear = retained_dates_df['DYear']
    retained_observed = retained_dates_df[f'{vi_name}_predicted_anomaly']
    outlier_dyear = outlier_dates_df['DYear']
    outlier_observed = outlier_dates_df[f'{vi_name}_predicted_anomaly']
    #
    plt.figure(figsize=(12, 4))
    #
    # plot retained dates
    plt.scatter(retained_dyear, retained_observed, marker='x', c='k', s=30,
                label=f'modeled observations ({len(retained_dyear)})')
    #
    # plot outlier observations
    plt.scatter(outlier_dyear, outlier_observed, marker='o', c='r', edgecolor='k', s=40,
                label=f'outlier observations ({len(outlier_dyear)})')
    #
    # plot zero line
    min_val = np.floor(np.min(all_dyear))
    max_val = np.ceil(np.max(all_dyear))
    plt.plot([min_val, max_val], [0.0, 0.0], 'k--', linewidth=1)
    #
    plt.legend(loc='upper left')
    #
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlabel(f'Date', fontsize=11)
    plt.ylabel(f'predicted {vi_name} anomaly', fontsize=11)
    title = f'Landsat {vi_name} predicted anomaly time series'
    plt.title(title, fontsize=12)
    plt.tight_layout()
    #
    plotfname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_time_series_anomalies.png'
    plotfpath = f'{output_path}/{plotfname}'
    plt.savefig(plotfpath, dpi=300, bbox_inches='tight')
    print(f'saved {plotfname}')
    return


print('Post-modeling summary')
print('Part 3: PLS prediction on original data')
print()
#
# pull needed metavals
output_path = metavals['output_path']
point_id = metavals['point_id']
vi_name = metavals['vi_name']
curve_str = metavals['vi_curve_str']
x_basis = metavals['x_basis']
optimal_outlier_round = metavals['optimal_outlier_round']
graph_output = metavals['graph_output']
#
# get original input dataset
full_dataset_fname = f'{point_id}_input_WxCD_VI_cleaned_trimmed.csv'
full_dataset_fpath = f'{output_path}/{full_dataset_fname}'
print(f'reading {full_dataset_fname}')
wxcd_vi_all_df = pd.read_csv(full_dataset_fpath, index_col=None)
all_dates = np.array(wxcd_vi_all_df['Date'])
all_doys = np.array(wxcd_vi_all_df['DOY'])
all_dyears = np.array(wxcd_vi_all_df['DYear'])
all_vi_values = np.array(wxcd_vi_all_df[vi_name])
n_all_dates = len(all_dates)
#
# get final detrending parameters for all variables
infname = f'{point_id}_pls_retained_{vi_name}_{curve_str}_{x_basis}_round_{optimal_outlier_round}_trend_params.csv'
infpath = f'{output_path}/{infname}'
outfname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_trend_params.csv'
outfpath = f'{output_path}/{outfname}'
os.system(f'cp {infpath} {outfpath}')
print(f'reading {outfname}')
trend_params_df = pd.read_csv(outfpath, index_col=None)
#
# get final standardizing parameters for all variables
infname = f'{point_id}_pls_retained_{vi_name}_{curve_str}_{x_basis}_round_{optimal_outlier_round}_stnd_params.csv'
infpath = f'{output_path}/{infname}'
outfname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_stnd_params.csv'
outfpath = f'{output_path}/{outfname}'
os.system(f'cp {infpath} {outfpath}')
print(f'reading {outfname}')
stnd_params_df = pd.read_csv(outfpath, index_col=None)
#
# get final set of outlier dates
outliers_fname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_all_outliers.csv'
outliers_fpath = f'{output_path}/{outliers_fname}'
print(f'reading {outliers_fname}')
vi_outliers_df = pd.read_csv(outliers_fpath, index_col=None)
outlier_dates = list(vi_outliers_df['Date'])
#
# get final n_components
infname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_PLSR_results.csv'
infpath = f'{output_path}/{infname}'
print(f'reading {infname}')
var_data_df = pd.read_csv(infpath, index_col=None)
regression_n_components = \
    var_data_df.loc[var_data_df.key==f'{vi_name}_round_{optimal_outlier_round}_final_n_components',
                    'value'].values[0]
#
# get final variables and betas
infname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_PLSR_vars+betas.csv'
infpath = f'{output_path}/{infname}'
print(f'reading {infname}')
var_data_df = pd.read_csv(infpath, index_col=None)
regression_vars = list(var_data_df['var'])
regression_nvars = len(regression_vars)
regression_betas = np.array(var_data_df['beta'])
print()
#
print(f'Predicting {vi_name} on all original image dates using final PLS model')
print(f'- {n_all_dates} dates')
print(f'- {regression_nvars} parameters')
print(f'- {regression_n_components} components')
print()
#
pls_prediction_df = pd.DataFrame({'Date': all_dates,
                                  'DOY': all_doys,
                                  'DYear': all_dyears,
                                  vi_name: all_vi_values})
for var_name in regression_vars:
    var_std_anoms = np.array(wxcd_vi_all_df[var_name])
    pls_prediction_df[var_name] = var_std_anoms
#
# add VI mean curve values and calculate residuals
all_vi_mean = [full_year_mean_vi[doy-1] for doy in all_doys]
pls_prediction_df[f'{vi_name}_fitted'] = all_vi_mean
all_vi_residuals = all_vi_values - all_vi_mean
pls_prediction_df[f'{vi_name}_residual'] = all_vi_residuals
#
#
regression_wxcd_detrended_stnd = np.zeros((n_all_dates, regression_nvars))
for n, var_name in enumerate(regression_vars):
    var_values = np.array(wxcd_vi_all_df[var_name])
    var_trend_params = np.array(trend_params_df[var_name][:2])
    var_detrended = detrend_given_stats(all_dyears, var_values, var_trend_params)
    var_stnd_params = np.array(stnd_params_df[f'{var_name}_detrended'])
    var_detrended_stnd = standardize_given_stats(var_detrended, var_stnd_params)
    regression_wxcd_detrended_stnd[:, n] = var_detrended_stnd
    pls_prediction_df[f'{var_name}_detrended_stnd'] = var_detrended_stnd
#
# calculate predicted VI residuals
predicted_vi_residuals_detrended_stnd = np.dot(regression_wxcd_detrended_stnd, regression_betas)
#
# destandardize + retrend predicted VI residuals
vi_residual_stnd_params = np.array(stnd_params_df[f'{vi_name}_residual_detrended'])
predicted_vi_residuals_detrended = \
    destandardize(predicted_vi_residuals_detrended_stnd, vi_residual_stnd_params)
vi_residual_trend_params = np.array(trend_params_df[f'{vi_name}_residual'][:2])
predicted_vi_residuals = \
    retrend(all_dyears, predicted_vi_residuals_detrended, vi_residual_trend_params)
pls_prediction_df[f'{vi_name}_residual_predicted'] = predicted_vi_residuals
predicted_vi_values = predicted_vi_residuals + all_vi_mean
pls_prediction_df[f'{vi_name}_predicted'] = predicted_vi_values
predicted_vi_anomalies = all_vi_values - predicted_vi_values
pls_prediction_df[f'{vi_name}_predicted_anomaly'] = predicted_vi_anomalies
outlier_bool = [int(date in outlier_dates) for date in all_dates]
pls_prediction_df[f'{vi_name}_outlier'] = outlier_bool
#
outfname = f'{point_id}_{vi_name}_{curve_str}_{x_basis}_final_PLS_prediction.csv'
outfpath = f'{output_path}/{outfname}'
pls_prediction_df.to_csv(outfpath, index=False)
print(f'wrote {outfname}')
#
if graph_output:
    pls_prediction_regression_plot(metavals, pls_prediction_df)
    # pls_prediction_time_series_plot(metavals, pls_prediction_df)
    pls_prediction_time_series_anomaly_plot(metavals, pls_prediction_df)


#
#

## **End of CANOPY-Phenology Model v0.99 in Jupyter Notebook format using Python v3.12**
